# Capstone Project - The Battle of Neighborhoods 
## 'The Best Walkable Spots for Students in NY'

### Final Report

## Introduction

In this first week, you have to clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

## Table of Contents

__1.Introduction - Business Problem Definition.__

__2. Data__

__3. Methodology__

__4. Results__

__5. Discussion__

__6. Conclusion__

# Introduction - Business Problem Definition

## 1. Study Context

There a category of students or young adults that would like to settle temporarily in New York City. This type of people has often low financial resources. They cannot afford to own and maintain a car with all the expenses that it can generate. Sometimes it is also because they want to adopt a greener transportation style to reduce their carbon footprint.

To help for their daily expenses or to supplement their basic income, they often hold jobs in the restaurants, shops, bars, etc. Moreover, as generally with young people, they also like to frequent the lively neighborhoods to meet their friends or other people. 

We can therefore predict that this category of population will be interested in looking for rental housing in lively areas allowing both leisure outings and opportunities to find odd jobs.

Of course, since they do not have a personal vehicle because lacking financial means or by the choice of ecological convictions, these students or young people will certainly look for renting nearby rooms or studios in neighborhoods as close as possible to places where they can easily find extra work.

A criterion of choice will be the proximity of the place of residence to these employment zones, having the possibility of getting there as quickly as possible first by walking as a pedestrian then using a skate-board, a bicycle or by public transport such as bus lines or metro etc ...

## 2. Problem Statement


As a Data Scientist, I would like to study the neighborhoods of New York and make a classification according to the number of places of shopping, outings and leisure and the accessibility of these places by walking (in a first study).

Later, we can refine by considering also other transportation means such as bike or public transport such as buses and metro.

Finally to be closer to the reality of these low-income people, we can add the rental price criterion (average in the neighborhood) to study the influence on the classification.

Nota: in this case study, the important factor of "income required per tenant" to be able to rent a room has not not been considered. Of course, it must be taken into account in a real life.

## 3. Target Audience

Who could be interested in this problem ?

At least we can list the following target populations :

1. Students that would like to find a job close to the place of residence

2. People who do not own any personal vehicle because they have either ecological convictions or low income but who love lively places with lots of restaurants, cafes, shops, cultural places etc. So those who are interested in living in a place with a high level walk-ability index.

3. Companies that work around search engines for rental accommodation agencies

4. Reception centers and help for students 

# Data Set Definition

A summary of needed data can be established from the problem definition above. 

As an output of the solution, we would like to provide a global picture of the New York places indicating where it is possible to rent a room or studio and with a score calculated based on the proximity of restaurants, bars, Coffee Shops or other businesses fully accessible to pedestrians. Something like a heat-map of places in New York City.

So we will need to retrieve the following information:

1. The list and localisation of New York neigborhoods
2. The list and localisation of places such as coffee shop, restaurants, miscellaneous shop, theaters etc around a specific place in New York 
3. The list of places with their walkability measure
4. The list of places with their average rental price

In the next sections, each data set is described given the origin, the type of data and the way it will be used to solve the problem we defined above.

##  1. New York neigborhoods

##  2. Venues in New York Neigborhoods

##  3. Walkability Measures of New York Places

##  4. Average Rental Price of New York Places

# Initialization and Dependencies
Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import os
import time

 # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge geopy --yes
# Use pip instead
#!pip3 install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#import pandas.api.CategoricalDtype
#from pandas.api.types import CategoricalDtype

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#µ!pip3 install folium0.5.0
import folium # map rendering library

# Install website scraping libraries and packages in Python from BeautifulSoup 
#!conda install -c conda-forge beautifulsoup4 --yes  # uncomment this line if you haven't completed 
from bs4 import BeautifulSoup as bs

print('Libraries imported.')

Some utility data and functions

In [ ]:
# Neede for filtering venues retrieval in Foursquare
target_root_categories = ['4d4b7105d754a06374d81259','4d4b7104d754a06370d81259',
                          '4d4b7105d754a06373d81259','4d4b7105d754a06376d81259',
                          '4d4b7105d754a06377d81259','4d4b7105d754a06378d81259'
                         ]

# Check if a file is existing
def not_present(filename):
    return not os.path.isfile('./'+ filename)


# Arrays to define values and order for Categorical columns 
ws_cat = ['Car-Dependant', 'Somewhat Walkable', 'Very Walkable', 'Walker’s Paradise']
budget_cat =['Cheap', 'Average', 'Pricey']

# Function to convert string array into categorical array
# Return a value to setup order for a categorical variable
def set_col_cat(df, catTab=budget_cat):
    raw_cat = pd.Categorical(df, categories=catTab, ordered=True)
    df = raw_cat
    return df
 
# Return a value to setup order for a categorical variable
def convertCatToNum(x,cat_type):
    val = None 
    if (cat_type == 'b'):  # budget category
        if x == 'Cheap':
            val = 2        # 1
        elif x == 'Average':
            val = 1     # 0
    elif cat_type == 'w':   # Walk Score category
        if x == 'Very Walkable':
            val = 1      #0
        elif x == 'Walker’s Paradise':
            val = 2      #1
    else:
        print('convertCatToNum : Invalid cat_type parameter', cat_type)
    
    return val

# Re-interpret/Rescale Budget label relative to price intervals
def rescaleBudgetLabel(br1):
    if(br1 >= 0 and br1 <= 2149):
        cat = 0
    elif (br1 <= 3149):
        cat = 1
    else :
        cat = 2
    return budget_cat[cat]
 
#ws_cat = ['Car-Dependant', 'Somewhat Walkable', 'Very Walkable', 'Walker’s Paradise'] 
# Re-interpret/Rescale label relative to Walk score intervals
def getWalkLabel(ws):
    cat = -1
    if(ws >= 0 and ws <= 49):
        cat = 0
    elif (ws <= 69):
        cat = 1
    elif (ws <= 89):
        cat = 2
    elif (ws <= 100):
        cat = 3
    else :
        return 'Unknown'

    return ws_cat[cat]

#p Calculate the sum of the row values 
def calc_total(row, ncol, dd):
    dd['Count'] = 1
    rowind = row.name
    for i in range(len(ws_cat)):
        val = round(dd.groupby(['Budget']).get_group(rowind).groupby(['WS_descr'])[ncol].sum()[i])
        row[i] = val
    return row

## 1. New York City Neighborhoods Data 

#### New York Neighborhoods

We know from a previous lab that New York city has a total of 5 boroughs and 306 neighborhoods. In order to segment the neighborhoods and explore them, we will essentially need the dataset mentioned above that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and longitude coordinates of each neighborhood.

This dataset exists at the link we have been given : https://geo.nyu.edu/catalog/nyu_2451_34572. 

To download the description file in json format, we have to run a wget command and access the data.

After that, we are able to parse the json features array of 306 items to extract all the information about the neighborhoods of New York.

This step leads us to build a dataframe, neighborhoods, containing the elementary data needed to solve our problem i.e. neighborhoods with location coordinates.

Using pandas library, we map the json data to the neighborhoods dataframe containing 4 columns listed below :

        Borough, Neighborhood, Latitude, Longitude

So let's go ahead and do that.

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

#### Load and explore the data

In [ ]:
with open('newyork_data.json') as json_data: 
    newyork_data = json.load(json_data)

Let's take a quick look at the data.
Display the 5 first elements.

In [ ]:
print ('Number of interesting data : ', newyork_data['totalFeatures'])
{k: newyork_data[k] for k in list(newyork_data)[:2]}

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [ ]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [ ]:
neighborhoods_data[0]

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [ ]:
neighborhoods

Then let's loop through the data and fill the dataframe one row at a time.

In [ ]:
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [ ]:
print("neighborhoods.shape = ", neighborhoods.shape)
#neighborhoods

And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

And also make sure that each neighborhood is uniquely defined : there are neighborhoods names duplicated as seen below .

In [ ]:
print('The dataframe has {} boroughs, {} neighborhoods but {} unique neighborhoods names.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0],
        len(neighborhoods['Neighborhood'].unique())
    )
)
print ('Number of neighborhood name duplicates =', 
      len(neighborhoods['Neighborhood']) - len(neighborhoods['Neighborhood'].unique()))

So, we will need to rename those names to be sure that each neighborhood is uniquely defined !

There are neighborhoods names duplicated as seen below .

In [ ]:
dup = neighborhoods[neighborhoods.duplicated(['Neighborhood'], keep=False)]
dup

So, rename those names by adding the 2 first letters of Borough to the neighborhood name
See below .

In [ ]:
df = dup
d_idx = df.index.values
print(d_idx)

for i, n_i in enumerate(d_idx):
    #print('i =', i, 'n_i', n_i, ' ->orig = ', neighborhoods.iloc[n_i,0], neighborhoods.iloc[n_i,1])
    n_name =  df.iloc[i,1]
    b_name = (str(df.iloc[i,0]))[0:3]
    neighborhoods.iloc[n_i,1] = n_name + ', ' +  b_name
    print(n_name, "---> renamed in : ", neighborhoods.iloc[n_i,1])


Now, we can consider the all 306 neighborhoods and try to get the most walkable areas in each of them relative to the higher density of target venues. 
1. get the area with the highest degree of venues (from fourthsquare)
2. measure walkability for each area
3. cluster according to walkability of each area

First Let's get the geographical coordinates of New York City so that we can create a map to visually locate neighborhoods in New York City.

For that we will use the _**geopy**_ library to get the coordinates and _**folium**_ library to build and display the map. 

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [ ]:
address = 'New York City, NY'
geo_agent = "ny_explorer"

geolocator = Nominatim(user_agent=geo_agent)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are : {}, {}.'.format(address, latitude, longitude))

#### Create a map of New York City with neighborhoods superimposed on top.

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Now that it is possible to locate the neighborhoods in New York City, we will explore each of them to compute the number of venues around them so that we can rank each neighborhood on a scale that measures the degree of opportunity (maximum of venues) to find a student job.

To explore the venues for each neighborhood we will use the _**Foursquare**_ API.

#### Define Foursquare Credentials and Version

In [ ]:
def buildCreds():
    # Manage credentials
    FSQ_CLIENT_ID = 'XXXXXXXXX' 
    FSQ_CLIENT_SECRET = 'YYYYYYYYY'
    WS_APIKEY1 = 'ZZZZZZZZ' # Paul
    WS_APIKEY2 = 'ZZZZZZZZ' # fafa
    WS_APIKEY3 = 'ZZZZZZZZ' # tofrou@yopmail.com 
    WS_APIKEY4 = 'ZZZZZZZZ' # mailboxy.fun

    cred_list =[FSQ_CLIENT_ID, FSQ_CLIENT_SECRET, WS_APIKEY1, WS_APIKEY2, WS_APIKEY3, WS_APIKEY4]
    cred_cols =['API_KEY']
    cred_df = pd.DataFrame([item for item in cred_list])
    cred_df.columns = cred_cols
    
    cred_df.to_csv('Credential_apis.csv')

    print ('Data file Credentials Saved !')

    return None
    

In [ ]:
# Try to load from local file system in case we did this before
fromFile = False
try:
    cred_df = pd.read_csv('Credential_apis.csv')
    cred_df.drop(cred_df.columns[[0]], axis=1, inplace=True)
    print('Credentials for apis data loaded.')
    fromFile = True
except:
    pass

# If not already loaded use the Foursquare API to get the data
if not fromFile:
    print ('Data file Credentials not existing! Build it')
    cred_df = buildCreds()
    
for i in range(cred_df.shape[0]):
    print ('i = ',i, ' -> key = ', 'got it')

In [ ]:
CLIENT_ID = cred_df.iat[0, 0]     # your Foursquare ID
CLIENT_SECRET = cred_df.iat[1, 0] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + 'got it!')
print('CLIENT_SECRET:' + 'got it!')

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

For each neighborhood in our dataframe, we will get the number of venues in a radius of     0.25 miles (~402 m).
This value of the radius has been chosen because we will consider the _"walkability"_ score of the neighborhood that will be computed later by _**Walk Score API**_ is based on this number.

As explained on the _**Walk Score**_ Web page :

" _**Walk Score Methodology**_ _*measures the walkability of any address using a patented system. For each address, Walk Score analyzes hundreds of walking routes to nearby amenities. Points are awarded based on the distance to amenities in each category. Amenities within a 5 minute walk (.25 miles) are given maximum points. A decay function is used to give points to more distant amenities, with no points given after a 30 minute walk.*_"

Of course we could have fragmented or paved the neighborhoods with a set of figures like 'circle, square, rectangle etc ..', get the center of each and calculate the number of venues.

Note  : Important information : we are limited in the number of API calls for my "Sandbox Account" that is Sandbox Account

    950 Regular Calls / Day
    50 Premium Calls / Day
    1 Photo per Venue
    1 Tip per Venue Hourly Rate Limit Overview
    An application can make a maximum of 5,000 userless requests per hour to venues/* endpoints.

An application can make a maximum of 5,000 userless requests per hour to venues/* endpoints.
So I have to take care about which information and number of calls I can perform to get what I need.
It is why I will try to get only the venues that could interest a student i.e. those that are 'good' candidate places to find a job. Most of the data on foursquare is food (restaurants) and nightlife relate.

#### Now, let's get the top 100 venues that are in each neighborhood within a radius of 400 meters that is less than 5 minute walk.

## 2. Foursquare Exploration of Venues  in New York City Neighborhoods 

#### Let's create a function to get the venues information for all neighborhoods 

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, categories="", radius=500):
    
    if categories == "":
        url_str = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'       
    else :
        url_str = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'

    venues_list=[]
    i = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
    #    print(i+1, " -> ", name)
            
        # create the API request URL
        if categories == "":
            url = url_str.format(CLIENT_ID, CLIENT_SECRET, VERSION, 
                                 lat, lng,
                                 radius, LIMIT)
        else :
            url = url_str.format(CLIENT_ID, CLIENT_SECRET, VERSION, 
                                 lat, lng, categories,
                                 radius, LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]
        #print ("response ->", results)
        results = results['groups'][0]
        #print ("groups[0] ->", results)
        results = results['items']
        print(i, " -> ", name, " -> venues number = ", len(results))
        i = i +1
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

print ("getNearbyVenues function defined !!!!")

#### Now the code to run the above function on each neighborhood and create a new dataframe called *newyork_venues*.

As we are limited by API calls, we will try to run the code once and store the result in a local file so that we can always continue the study without this limitation.

In [ ]:
radius = 500
LIMIT = 100

# Try to load from local file system in case we did this before
fromFile = False
try:
    newyork_venues = pd.read_csv('NYC_Neighborhoods_venues.csv')
    newyork_venues.drop(newyork_venues.columns[[0]], axis=1, inplace=True)
    print('NYC Neighborhoods Venues data loaded.')
    fromFile = True
except:
    pass

# If not already loaded use the Foursquare API to get the data
if not fromFile:
    print ('Data file for venues not existing ! Call API')
    newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                     latitudes=neighborhoods['Latitude'],
                                     longitudes=neighborhoods['Longitude']
                                     )
    newyork_venues.to_csv('NYC_Neighborhoods_venues.csv')

We can now examine the retrieved venues.

print ('Number of retrieved venues is : ', newyork_venues.shape[0])
newyork_venues.head(20)

10243 retrieved venues !

Well seems good !
We can see that most part of the venues are good candidates to places where a student can find a job but some of them like "Trail" need to be filtered out and removed from the final result.
For the moment we let them ...

Let's see the number of venues grouped by neighborhood.

In [ ]:
df = newyork_venues.groupby('Neighborhood').count() #, as_index=True
df.sort_values('Venue',ascending=False)


In [ ]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

Ok let's fisrt filter with the target venues root categories [OK,NOK]
From the following list of root categories :

#### Use API to Search for 50 top venues filtered by category type :

 . Arts & Entertainment : 4d4b7104d754a06370d81259            -> _**OK**_
 
 . College & University : 4d4b7105d754a06372d81259            -> _**NOK**_

 . Event : 4d4b7105d754a06373d81259                           -> _**OK**_

 . Food : 4d4b7105d754a06374d81259                            -> _**OK**_

 . Nightlife Spot : 4d4b7105d754a06376d81259                  -> _**OK**_
 
 . Outdoors & Recreation : 4d4b7105d754a06377d81259           -> _**OK**_ 
 
 . Professional & Other Places : 4d4b7105d754a06375d81259     -> _**NOK**_

 . Residence : 4e67e38e036454776db1fb3a                       -> _**NOK**_
 
 . Shop & Service : 4d4b7105d754a06378d81259                  -> _**OK**_
 
 . Travel & Transport : 4d4b7105d754a06379d81259              -> _**NOK**_
 
 Of course we could discuss about this choice, given that sometimes sub-categories are or are not relevant even in not chosen root categories.


In [ ]:
# define function to format venues address
# Lot of problems with missing fields ex: 'address' or 'city'
# -> so take formattedAddress
def get_address(flocation):
    # print ('location(address) = ', location['formattedAddress']) #['address'], location['city'])
    address = flocation[0] + ', ' + flocation[1]
    return address

# build the list of chosen categories (easier to find a job)
root_categories = target_root_categories[0]
for cat in  target_root_categories[1:]:
    root_categories = root_categories + ',' + cat

# build the dataframe of selected categories of venues
# Normally, we are limited by the API to 50 retrieved venuees
def getNearbyFilteredVenues(names, latitudes, longitudes, categories="", radius=250, limit=50):
    # Use search API with the list of selected root categories
    url_str = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&intent=browse&categoryId={}&ll={},{}&radius={}&limit={}"

    # Retrieve list of target venues in a radius of 500 m from the coordinates
    # of each of the 306 identified neighborhoods and complying the given categories
    venues_list=[]
    i = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(i, " -> ", name)
        time.sleep(0.3)
        url = url_str.format(CLIENT_ID, CLIENT_SECRET, VERSION,
                             categories,lat, lng, 
                             radius, 50)
        try:
            results = requests.get(url).json()["response"]["venues"]
        except:
            print(i, " -> ", name, " -> No venues !!!!")
            continue
            pass

        print(i, " -> ", name, " -> venues number = ", len(results))

        # For each resulting venue list for this neighborhood
        # build the associated data : name address coordinates etc. 
        for v in results:
            #parsed_venue = json.loads(venue)
            #print("->",json.dumps(v, indent=4, sort_keys=True))
            #
            venues_list.append([(
                name,                       # neighborhood name
                lat,                        # neighborhood latitude
                lng,                        # neighborhood longitude
                v['name'],                  # venue name
                get_address(v['location']['formattedAddress']), # venue address
                v['location']['lat'],       # venue latitude
                v['location']['lng'],       # venue longitude
                v['categories'][0]['name']  # venue category
            )])
    
        i = i +1

    # Once we have registered all retrieved venues, put them in a new filtered dataframe
    nearby_filtered_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_filtered_venues.columns = ['Neighborhood',
                                      'Neighborhood Latitude',
                                      'Neighborhood Longitude',
                                      'Venue',
                                      'Address',
                                      'Venue Latitude',
                                      'Venue Longitude',
                                      'Venue Category']

    return nearby_filtered_venues

print("Filtered Venues Functions defined !")

In [ ]:
# Try to load from local file system in case we did this before
fromFile = False
try:
    newyork_filtered_venues = pd.read_csv('NYC_Neighborhoods_filtered_venues.csv')
    newyork_filtered_venues.drop(newyork_filtered_venues.columns[[0]], axis=1, inplace=True)
    print('NYC Neighborhoods Filtered Venues data loaded.')
    fromFile = True
except:
    pass

# If not already loaded use the Foursquare API to get the data
if not fromFile:
    print ('Data file for filtered venues not existing ! Call API')
    newyork_filtered_venues = getNearbyFilteredVenues(names=neighborhoods['Neighborhood'],
                                                      latitudes=neighborhoods['Latitude'],
                                                      longitudes=neighborhoods['Longitude'],
                                                      categories=root_categories,
                                                      radius=500, 
                                                      limit=50
                                                      )
    # Export the dataframe values to a local file  
    newyork_filtered_venues.to_csv('NYC_Neighborhoods_filtered_venues.csv')

print ('Number of retrieved filtered venues is : ', newyork_filtered_venues.shape[0])
newyork_filtered_venues[0:10]

 11311 retrieved venues (previously it was 13653 ???) !

Well it seems that i got more filtered venues that for the first call without filtered request !
In fact, each API call retrieves a different number of venues because updated. 
So let's take this number for now.
Let's see the number of venues grouped by neighborhood.

In [ ]:
df = newyork_filtered_venues.groupby('Neighborhood', as_index=False).count() #
df = df.sort_values('Venue',ascending=False).reset_index(drop=True)
df.head()

In [ ]:
print('There are {} uniques filtered categories.'.format(len(newyork_filtered_venues['Venue Category'].unique())))

Let see how many Neighborhoods cumulate the maximum number of 5000 allowed per day by Walk API to compute the walkability of places.

In [ ]:
df['Venues CumSum'] = df['Venue'].cumsum()
df.head()

Now we know that the first 100 elements in range [0, 99] (Lenox Hill -> Bay Terrace, Sta) cumulate from 0 to 5000 venues.

Neighborhoods in range [100, 225]  (Canarsie -> Roosevelt Island) cumulate from 5001 to 9986 venues

Last range [226,305] (Morris Park-> Jamaica Estates) cumulate from 9987 to 1131 venues

Most part of neighborhoods cumulate more than 50 retrieved venues but limited to 50 cause API limitation.

**98 		Carroll Gardens 	50 	50 	50 	50 	50 	50 	50 	4950 **

**99 		Bay Terrace, Sta 	50 	50 	50 	50 	50 	50 	50 	5000 **

**100 		Canarsie 			50 	50 	50 	50 	50 	50 	50 	5050 **

We can now compute the walkability for these neighborhoods in this order i.e.

- loop on neighborhoods of NYC
   - loop on each venue
     - compute walkability 
     - add walk/bike/transit score values in new columns of newyork_filtered_venues
   - compute mean values by neighborhood if needed
  
Discussion :
 - we can perform the call for each of the 11311 retrieved venues (need at leat 3 days)
 - we can just perform the walkability calculation on the first 5000 venues but 143 neighborhoods have 50 venues
 - we can pick a number of venues according to the density of retrieved venues for each neighborhood so that
   all neighborhoods are covered and can be compared in term of walkability

##  3. Walkability Scores of New York Venues

### Walk Scores

Now that we have our location candidates, let's use Walk Score API to get info on venues in each neighborhood.

So it is now possible to rank each venue, but limit is 5,000 calls by day for a Free Version. So as I have to choose the targets carefully : 
 - because not sure that just take the neighborhoods with the most venues is the best strategy ??? 
 - Normally should consider the "best venues" to find a job across all neighborhood  -> so filter by venue type ???

Add each row in a new dataframe until 5000 venues is reached.

### a) Walk Score API format

A call to the Walk Score API follows the format below :

> `http://api.walkscore.com/score?format=json&address=`**ADDRESS**&lat=**LATITUDE**&lon=**LONGITUDE**&transit=1&bike=1&wsapikey=**WSAPIKEY**

Just take a look at the existing data to see how to add the walking scores.

In [ ]:
newyork_filtered_venues.head(2)

### b) Walk Score GET request URL

Following the format given above, thanks to the function below, I built the url from the parameters identifying a venue.    

In [ ]:
def get_url(address, lat, lon, wsapikey=""):
    url = 'http://api.walkscore.com/score?format=json&address={}&lat={}&lon={}&transit=1&bike=1&wsapikey={}'.format(
            address, 
            lat,
            lon,
            wsapikey)
    return url

### c) Walk Score Retrieval Functions Definition

The functions below retrieve the walk scores and build the new dataframe with the added scores for each venue.

In [ ]:
# get the score of venues.
# Normally, we are limited by the API to 5000 calls per day
# An example of the results from WS API that we have to process
result_str = '{\
    "bike": {\
        "description": "Bikeable",\
        "score": 54\
    },\
    "description": "Somewhat Walkable",\
    "status": 1,\
    "walkscore": 59\
}'


def getScore(v_name, lat, lon, adr):
    url = get_url(adr, lat, lon, wsapikey=WSAPIKEY)
    scores_descs = [-1, "", -1, "", -1, ""]
    results = requests.get(url).json() # json.loads(result_str) # 
    status = results['status']
    
    bscore = tscore = -1
    bdescr = tdescr = ""
    if (status != 1):
        print("Venue :", v_name, " -> Error status returned = ", status)
    else :
        #print("->",json.dumps(results, indent=4, sort_keys=True))
        wscore, wdescr = results['walkscore'],results['description']
        try:
            bscore, bdescr = results['bike']['score'], results['bike']['description']
        except:
            pass
        try:
            tscore, tdescr = results['transit']['score'], results['transit']['description']
        except:
            pass

        scores_descs = [wscore, wdescr, bscore, bdescr, tscore, tdescr]

    return scores_descs

# Neighborhood Latitude 	Neighborhood Longitude 	Venue 	Venue Latitude 	Venue Longitude 	Venue Category
def getWalkScoreVenue(venues_score_df,beg=0):
    # Retrieve list of target venues in a radius of 500 m from the coordinates
    # of each of the 306 identified neighborhoods and compute walk score
    
    venues_df_ncol = len(venues_score_df.columns)
    for index, row in venues_score_df.iterrows():
        name = row['Venue']
        lat = row['Venue Latitude']
        lng = row['Venue Longitude']
        adr = row['Address']
        
        #time.sleep(0.001)
        results = getScore(name, lat, lng, adr)
        i = 0
        col= venues_df_ncol - 6
        print('index = ', index, ' Venue =', name, 'col =',col, 'results ->', results)
        r_num = index - beg
        venues_score_df.iat[r_num, col] = results[i] # wscore
        i += 1
        col +=1 
        venues_score_df.iat[r_num, col] = results[i] # wsdescr
        i += 1
        col +=1 
        venues_score_df.iat[r_num, col] = results[i]  # bscore
        i += 1
        col +=1 
        venues_score_df.iat[r_num, col] = results[i] # bdescr
        i += 1
        col +=1 
        venues_score_df.iat[r_num, col] = results[i] # tscore
        i += 1
        col +=1 
        venues_score_df.iat[r_num, col] = results[i] # tdescr
#    return venues_score_df

# Try to load from local file system in case we did this before
def load_scores(df_venues, beg, end):
    filename = 'NYC_venues_walkability_{}_{}.csv'.format(beg,end)
    fromFile = False
    try:
        df_venues = pd.read_csv(filename)
        df_venues.drop(df.columns[[0]], axis=1, inplace=True)
        print('NYC Neighborhoods Venues Walkability Scores data loaded from file :', 
              filename)
        df_venues.Walk_descr = set_col_cat(df_venues.Walk_descr, ws_cat)
        fromFile = True
    except:
        pass

    # If not already loaded : use the Foursquare API to get the data
    if not fromFile:
        print ('Data file for filtered venues not existing ! Call API')
        # Add scores columns for walkability and bike
        df_venues = pd.concat([df_venues,
                        pd.DataFrame(columns=['Walk_score', 'Walk_descr',
                                              'Bike_score', 'Bike_descr',
                                              'Transit_score', 'Transit_descr'])],
                       sort=False)
        getWalkScoreVenue(df_venues,beg)
        df_venues.Walk_descr = set_col_cat(df_venues.Walk_descr, ws_cat)

        # Export the dataframe values to a local file  
        #print(df)
        df_venues.to_csv(filename)
        print ('Venues with Walk Scores Data file created : ', filename)
    return df_venues

### d) Walk Scores Retrieval

Now, let's create the new dataframe from the venues extended with walk, bike, transit scores.

It was difficult to do all retrievals in one operation cause the limit imposed by the Walk Score APIs (less than 5000 calls per days).
So that I have to wait the next day to get the 5000 scores for the next 5000 venues.

Also sometimes, the Python machine stops without any feedback perhaps due to the allocated machine's capacity exceeded.
When that happens, the operation must be performed again from the beginning.

It is why, I splitted the calls in several steps to get the scores for all venues the 11311 venues.

In [ ]:
i=2
WSAPIKEY1 = cred_df.iat[i, 0] # Paul
i=i+1
WSAPIKEY2 = cred_df.iat[i, 0] # fafa
i=i+1
WSAPIKEY3 = cred_df.iat[i, 0] # tofrou@yopmail.com
i=i+1
WSAPIKEY4 = cred_df.iat[i, 0] # mailboxy.fun -> OK

### e. Data Processing : First Step

#### 1) Data Merging

As already said, I have splitted the calls in several steps to get the scores for all venues the 11311 venues.

I have now to merge all dataframes and files into one unique dataframe and file to consider the totality of the information on the venues for our analysis.


In [ ]:
# Try load from existing files and concatenate the 3 tables if any
try:
    df_ws_0_4999 = pd.read_csv('NYC_venues_walkability_0_4999.csv')
    df_ws_5000_9999 = pd.read_csv('NYC_venues_walkability_5000_9999.csv')
    df_ws_10000_plus =  pd.read_csv('NYC_venues_walkability_10000_plus.csv')
    df_venues_with_wscores = (df_0_4999.append(df_5000_9999)).append(df_ws_10000_plus)
    df_venues_with_wscores.drop(df_venues_with_wscores.columns[[0]], axis=1, inplace=True) 
    print('NYC Neighborhoods with all Venues Walkability Scores data loaded from basic files !')
except:
    print('Could not load NYC Neighborhoods with all Venues Walkability Scores from basic files !')
    pass    

In [ ]:
# Try to load walk scores from local file system in case we already created it in a previous step
fromFile = False
try:
    df_venues_with_wscores = pd.read_csv('NYC_venues_walkability_0_11310.csv')
    df_venues_with_wscores.drop(df_venues_with_wscores.columns[[0]], axis=1, inplace=True)
    print('NYC Neighborhoods with all Venues Walkability Scores data loaded.')
    fromFile = True
except:
    pass

# If not already loaded redo all work at steps above (using Walk Score API to get the data)
if not fromFile:
    print ('Data file of Venues with Walk Scores not existing ! Back Call API')

print ('Number of retrieved filtered venues is : ', df_venues_with_wscores.shape[0])
df_venues_with_wscores.head()

#### 2) Data Cleaning

Now take a look at the data and see if all information are ready to be used for our analysis !!!!

Clean if any missing data : adopt a strategy !
 - drop the row
 - replace the value by the same value of the surrounding venue with the same latitude and longitude
 - replace the value by the min of values of the surrounding venues
 - replace the value by the mean of values of the surroundingvenues 
 - etc.

So how many venues have empty walk scores (equal to -1) ?

In [ ]:
list_empty_wscores = df_venues_with_wscores.index[df_venues_with_wscores['Walk_score'] == -1].tolist()
print('Number of rows with empty Walk Score (=-1) = ', len(list_empty_wscores), 
      '\nlist = ', list_empty_wscores, '\n********\n') #, df_venues_with_wscores.loc[df_venues_with_wscores['Walk_score'] == -1]['Venue'])
for index_venue in list_empty_wscores:
    lat = df_venues_with_wscores.loc[index_venue]['Venue Latitude']
    lng = df_venues_with_wscores.loc[index_venue]['Venue Longitude']
    name = df_venues_with_wscores.loc[index_venue]['Venue']
    print ("Index = {0:5d} at coordinates : {1:3.15f}, {2:3.15f} -> Venue : {3}".format(index_venue, lat, lng, name))

Now we have the number and identification of the venues without walk score : 32 venues without walk scores.

This number represents only 32/11311 = 0,000972505 ~ 0.1%

So I think, it does not matter if this venues are removed before analysis.

So let's drop these rows and we should get a new dataframe with (11311 - 32 =) 11279 rows.

Quick check they have been removed.

In [ ]:
df_venues_cleaned_wscores = df_venues_with_wscores.drop(list_empty_wscores)
df_venues_cleaned_wscores.reset_index(drop=True, inplace=True)
print (df_venues_cleaned_wscores.shape[0])

for index_venue in list_empty_wscores[0:5]:
    name = df_venues_cleaned_wscores.loc[index_venue]['Venue']
    print ("Index = {0:5d} -> Venue : {1}".format(index_venue, name))

df_venues_cleaned_wscores.to_csv("NYC_venues_cleaned.csv")
print ("NYC_venues_cleaned.csv created !")
#print(df_venues_cleaned_wscores.head(), df_venues_cleaned_wscores.tail())

In [ ]:
# Try to load walk scores from local file system in case we already created it in a previous step
fromFile = False
try:
    df_venues_with_wscores = pd.read_csv('NYC_venues_walkability_0_11310.csv')
    df_venues_with_wscores.drop(df_venues_with_wscores.columns[[0]], axis=1, inplace=True)
    print('NYC Neighborhoods with all Venues Walkability Scores data loaded.')
    fromFile = True
except:
    pass

# If not already loaded redo all work at steps above (using Walk Score API to get the data)
if not fromFile:
    print ('Data file of Venues with Walk Scores not existing ! Back Call API')

print ('Number of retrieved filtered venues is : ', df_venues_with_wscores.shape[0])
df_venues_with_wscores.head()

##  4. Aggregation of Average Rental Price Data

As already said in the Data section, these measures will help to refine choice of places in New York neighborhoods that are the most suitable for people with low incomes such as the students population.
 
We will use the Web Site _**"renthop"**_ (https://www.renthop.com/) to get the page _https://www.renthop.com/average-rent-in/new-york-city-ny_ where are displayed the _**"Rental Stats and Trends"**_. This is a collection of information that gives:

1. *Historical Prices and Trends* : curves of the rental prices of housing by categories (studios, 1 bedroom or 2) since 3 years

2. *Median Rents* : table showing prices broken by housing category and quartiles (bottom 25%, median and top 25%)

3. *Average Rents by Neighborhoods* : table of average prices broken by "Neighborhoods" and given, by housing categories, a price and a classification of the needed budget (Cheap, Average, Pricey).

We are mainly interested by the data stored into the table *Average Rents by Neighborhoods*.

To extract the average prices by neighborhoods and by housing categories, I will follow steps explained below :

1. Download the HTML file at the given link : _'https://www.renthop.com/average-rent-in/new-york-city-ny'_ 

2. Register the file locally in _'NYC_renthop.html'_ 

3. Open the file and using _'BeautifulSoup'_ library, retrieve :
> - the table with the title as displayed above *"Average Rents by Neighborhoods"*
> - iterate through the next HTML elements to extract the name of the columns 
> - iterate through the next HTML elements to extract the values for each 'Neighborhood' row 

4. Build the _pandas Dataframe_ with the retrieved data

Now lets do what we have presented above : parse the HTML file thanks to the _'BeautifulSoup'_ library.

In [ ]:
!wget -q -O 'NYC_renthop.html' 'https://www.renthop.com/average-rent-in/new-york-city-ny'
print('HTML NYC Renthop page downloaded!')

#### Load the data

Next, let's load the data.

In [ ]:
# Get HTML content
with open("NYC_renthop.html") as fp:
    soup = bs(fp, 'lxml')

# Get the HTML table codes
#tagTable = soup.find('table', attrs={})

allTables = soup.findAll('table')
#print (len(allTables))

avgTable = allTables[1]

#Get table body for the 'Average Rents by Neighborhoods' table
bodyAvg = avgTable.thead
print('thead ===', bodyAvg)

print('\n***********************\n')
# Get 'Median Rents' data table
medianTable = allTables[0]

#Get table body for the 'Median Rents' table
bodyMedian = medianTable.thead
print('thead ===', bodyMedian)


#### Tranform the data into a *pandas* dataframe

The task is essentially transforming this HTML data  into a *pandas* dataframe.
So let's start by creating an empty dataframe with just the column names

In [ ]:
# Define the dataframe columns 
# get table column names -> all 'th' tags of the body in 'tr' fields
def getColNames(body, n):
    colTab = body.find_all('th')
#print (colTab)
    colNames = [(bs(str(colTab[i]),"lxml")).find('th').string.strip() for i in range(n)]
    print(colNames)
    return colNames

avgColNames = getColNames(bodyAvg,5)
medianColNames = getColNames(bodyMedian,4)

Then let's loop through the data and fill the dataframe one row at a time.

In [ ]:
# extract all 'tr' tagged fields to get table contents for each neighborhood
rentTab= avgTable.find_all('tr')
print(rentTab[0:2])
print('\n***********************\n')

medianTab = medianTable.find_all('tr')[2:]
print(medianTab)

Rental amounts strings have to be formatted in numbers like below :

**'$4,542 -> 4542'**


In [ ]:
def convertCurrency(curStr):
    val = -1
    if (curStr is not None):
        val = int(curStr.replace('$','').replace(',',''))
    return val

In [ ]:
# instantiate the dataframe with retrieved column names
renthop_df = pd.DataFrame(columns=avgColNames)

for n, rent in enumerate(rentTab):
    # Average Rents for the n.th Neighborhood name or link 
    # do 
    # for each element code 
    tabTd = rent.find_all('td')
    neighborhood = tabTd[0].string # first process neighborhood
    budget = tabTd[4].string # first process neighborhood

    # for all amounts 
    tabc = [None] * 3
    for i, value in enumerate(tabTd[1:4]):
         tabc[i] = convertCurrency(value.string)
#       #print('i = ', i , value.string)

    price_studio = tabc[0]
    price_1BR = tabc[1]
    price_2BR = tabc[2]
    
    # insert the built postal code into the dataframe Neighborhood 	Studio 	 	2BR 	Budget
    renthop_df = renthop_df.append({'Neighborhood' : neighborhood,
                            'Studio' : price_studio,
                            '1BR': price_1BR,
                            '2BR': price_2BR,
                            'Budget': budget},
                           ignore_index=True)

# Combine rows with same postal code into one row with the neighborhoods separated with a comma 
#df = postcode_df.groupby('Postcode', as_index=False).agg({'Borough':'first', 'Neighbourhood':', '.join})
#renthop_df.loc[0,'Budget']
renthop_df.Budget = set_col_cat(renthop_df.Budget)
renthop_df.head()

In [ ]:
budget_cat = renthop_df['Budget'].unique()
print('There are {} unique Budget categories.\n  -> {}'.format(len(budget_cat), budget_cat))

In [ ]:
# instantiate the dataframe with retrieved column names
medianRent_df = pd.DataFrame(columns=medianColNames)

for n, mrent in enumerate(medianTab):
    # Average Rents for the n.th Neighborhood name or link 
    # do 
    # for each element code 
    tabTd = mrent.find_all('td')
    beds = tabTd[0].string # first process neighborhood
    # for all amounts 
    tabc = [None] * 3
    for i, value in enumerate(tabTd[1:]):
         tabc[i] = convertCurrency(value.string)

#   #print ("n = ", tabc)
    bot25 = tabc[0]
    median = tabc[1]
    top25 = tabc[2]
    
    # insert the built postal code into the dataframe Neighborhood 	Studio 	 	2BR 	Budget
    medianRent_df = medianRent_df.append({medianColNames[0] : beds,
                                          medianColNames[1] : bot25,
                                          medianColNames[2] : median,
                                          medianColNames[3] : top25},
                                         ignore_index=True)

medianRent_df
#df[df.columns[1:]].replace('[\$,]', '', regex=True).astype(int)

In [ ]:
# Store data in files for future usage
filename = "NYC_Average_Rents.csv"
if not_present(filename):
    renthop_df.to_csv(filename)
    print (filename, " : created !")

filename = "NYC_Median_Rents.csv"
if not_present(filename):
    medianRent_df.to_csv(filename)
    print (filename, " : created !")


Looking good for the retrieved Average & Median Rents for registered neighborhoods.

But we have found that NYC has 5 boroughs and 306 neighborhoods and here we have retrieved data for only 68 neighborhoods.

We have to complete the missing values for all neighborhoods.

I decided to get the "Median Rents" values from the same data source to complete the missing values !

Let's go there...

Below it can be used once the files have been already generated in the steps above. 

In [ ]:
# Try to load walk scores from local file system in case we already created it in a previous step
fromFile = False
try:
    renthop_df = pd.read_csv('NYC_Average_Rents.csv')
    renthop_df.drop(renthop_df.columns[[0]], axis=1, inplace=True)
    renthop_df.Budget = set_col_cat(renthop_df.Budget)
    medianRent_df = pd.read_csv('NYC_Median_Rents.csv')
    medianRent_df.drop(medianRent_df.columns[[0]], axis=1, inplace=True)
    print('Loading NYC Average and Median Rents Data files Successful ! Skip BeautifulSoup !')
    fromFile = True
except:
    pass

# If not already loaded redo all work at steps above (using Walk Score API to get the data)
if not fromFile:
    print ('Cannot load NYC Average and Median Rents Data files ! Use BeautifulSoup !')

In [ ]:
# Neighborhood 	Studio 	1BR 	2BR 	Budget
list_empty_avgrent = renthop_df.index[renthop_df['Studio'] == -1].tolist()
print('Number of rows with empty rent value (=-1) = ', len(list_empty_avgrent), 
      '\nlist = ', list_empty_avgrent, '\n********\n') #
for index_avgrent in list_empty_avgrent:
    studio = renthop_df.loc[index_avgrent]['Studio']
    one_BR = renthop_df.loc[index_avgrent]['1BR']
    name = renthop_df.loc[index_avgrent]['Neighborhood']
    budget = renthop_df.loc[index_avgrent]['Budget']
    print ("Index = {0:5d} -> Studio = {1:5d} ,1BR = {2:5d}, Budget = {3:8s} -> Name = {4}".
           format(index_avgrent, int(studio), int(one_BR), budget, name))

So, we have found 29 on 68 neighborhoods that have no price for 'Studio' category.

We have to complete the missing values for all neighborhoods using the 'Budget' categories (['Average' 'Cheap' 'Pricey']).

I decided to get the "Median Rents" values from the same data source to complete the missing values according to budget category. If the category is :
- 'Cheap' then use 'Bot25%' value
- 'Pricey' then use 'Top25%' value
- 'Average' then use 'Median' value 

In [ ]:
# Neighborhood 	Studio 	1BR 	2BR 	Budget
def setMissingAvg(colName, rowIdx):
    list_empty_avgrent = renthop_df.index[renthop_df[colName] == -1].tolist()
    print('Number of rows with empty rent value (=-1) in colname {} = {}'.format(colName, len(list_empty_avgrent)), 
      '\nlist = ', list_empty_avgrent, '\n********\n') #
    
    rent_bot = medianRent_df.loc[rowIdx]['Bot 25%']
    rent_med = medianRent_df.loc[rowIdx]['Median']
    rent_top = medianRent_df.loc[rowIdx]['Top 25%']

    for index_avgrent in list_empty_avgrent:
        budget = renthop_df.loc[index_avgrent]['Budget']
        if (budget == 'Cheap'):
            renthop_df.ix[index_avgrent, colName] = rent_bot
        elif (budget == 'Average'):
            renthop_df.ix[index_avgrent, colName] = rent_med
        else:
            renthop_df.ix[index_avgrent, colName] = rent_top

setMissingAvg('Studio', 0)
setMissingAvg('2BR', 2)

renthop_df.head(20)

Let's examine the generated values. it seems that they are not correctly set. 

We should think to a better approximation method.

As example, a simple approximation algorithm could be : 

- For each row where value of Average[Studio] == -1 
- Base on Budget 
     -> find Median[1BR] 
     -> define Ratio = (Average[row][1BR] / Median[1BR])
     -> define Average[row][Studio] = Median[Studio] * Ratio

Let's try again.
We have to complete the missing values for all neighborhoods using the 'Budget' categories (['Average' 'Cheap' 'Pricey']).


In [ ]:
# Neighborhood 	Studio 	1BR 	2BR 	Budget
def setMissingAvgRatio(colName, rowIdx):
    list_empty_avgrent = renthop_df.index[renthop_df[colName] == -1].tolist()
    print('Number of rows with empty rent value (=-1) in colname {} = {}'.format(colName, len(list_empty_avgrent)), 
      '\nlist = ', list_empty_avgrent, '\n********\n') #
    
    rent_bot = medianRent_df.loc[rowIdx]['Bot 25%'] # 'Cheap'
    rent_med = medianRent_df.loc[rowIdx]['Median']  # 'Average'
    rent_top = medianRent_df.loc[rowIdx]['Top 25%'] # 'Pricey'

    for index_avgrent in list_empty_avgrent:
        # compute a ratio using '1BR' references
        budget = renthop_df.loc[index_avgrent]['Budget']
        if (budget == 'Cheap'):
            val = (renthop_df.ix[index_avgrent, '1BR'] /
                   medianRent_df.loc[1]['Bot 25%']) * rent_bot
        elif (budget == 'Average'):
            val = (renthop_df.ix[index_avgrent, '1BR'] / 
                   medianRent_df.loc[1]['Median']) * rent_med
        else:
            val = (renthop_df.ix[index_avgrent, '1BR'] /
                   medianRent_df.loc[1]['Top 25%']) * rent_top
        
        # Apply the ratio to the colName value
        val = round(val)
        #print ('Current = {} -> corrected = {}'.
        #       format(renthop_df.ix[index_avgrent, colName],
        #              val))
        renthop_df.ix[index_avgrent, colName] = val
    
# Try to load rental data from local file system in case we already created it in a previous step
fromFile = False
try:
    renthop_df = pd.read_csv('NYC_Avg_Rents_Filled.csv')
    renthop_df.drop(renthop_df.columns[[0]], axis=1, inplace=True)
    renthop_df.Budget = set_col_cat(renthop_df.Budget)
    fromFile = True
    print('Loading NYC Average filled Data file Successful !')
except:
    pass

# If not already loaded redo all work at steps above (using Walk Score API to get the data)
if not fromFile:
    print ('Cannot load NYC Average Filled and Median Rents Data files ! Using function !')
    try:
        renthop_df = pd.read_csv('NYC_Average_Rents.csv')
        renthop_df.drop(renthop_df.columns[[0]], axis=1, inplace=True)
        renthop_df.Budget = set_col_cat(renthop_df.Budget)
        
        medianRent_df = pd.read_csv('NYC_Median_Rents.csv')
        medianRent_df.drop(medianRent_df.columns[[0]], axis=1, inplace=True)
        print('Loading NYC Average filled and Median Rents Data files Successful !')
        setMissingAvgRatio('Studio', 0)
        setMissingAvgRatio('2BR', 2)
        # Store these data
        renthop_df.to_csv('NYC_Avg_Rents_Filled.csv')
        print('Saving NYC Average filled file Successful !')
    except:
        print('Cannot load NYC Average and/or Median Rents Data file!\n',
              'Apply steps to rebuild data !')
        pass


renthop_df.head()

This looks better (although we can continue to discute a more accurate correction method of course !). But for the moment we can go forward and add this data to the venues and see how we can achieve our goal.

The problem we face now is also missing data for all NYC neighborhoods.
We have only 68 neighborhoods out of 306 filled with rental data.
Try to match the neighborhoods of the 2 sets.
It seems we cannot match 34 over 68 neighborhoods from rental data to the NYC neighborhoods in the venues dataframe !!!

In [ ]:
print ('Total neighborhoods in NYC = {} , neighborhoods with rental data = {}'.
       format(neighborhoods.shape[0], renthop_df.shape[0]))
df_ws = df_venues_cleaned_wscores.drop(['Bike_score', 'Bike_descr', 'Transit_score','Transit_descr'], axis=1)
df_rent = renthop_df.drop(['Studio','2BR'], axis=1)
ws_nghrd_list = list(df_ws['Neighborhood'].unique())
rent_nghrd_list  = list(renthop_df['Neighborhood'])

df_rent.head()

#### Use geopy library to get the latitude and longitude values of rental neighborhoods.

Our user_agent <em>ny_explorer</em> is reused as shown below to get localisation coordinates for the neighborhoods contained in rental data.
Once localisation has been completed, clean and handle missing data if any.

In [ ]:
address_NYC = ', New York City, NY'
geo_agent = "ny_explorer"
geolocator = Nominatim(user_agent=geo_agent)

latitude = []
longitude = []
unknown = 0
# 'Neighborhood Latitude'  'Neighborhood Longitude'
for address in rent_nghrd_list :
    #time.sleep(0.300)
    #print ("Address is : ", address)
    try:
        location = geolocator.geocode(address + address_NYC)
        lat = location.latitude
        lng = location.longitude
    except:
        lat = -1
        lng = -1
        #print('  -> Cannot load NYC coordinate address!')
        unknown += 1
        pass
    
    latitude.append(lat)
    longitude.append(lng)
#    print('   -> The geograpical coordinate of {} are : {}, {}.'.format(address, lat, lng))
print ('Unknown = ', unknown)

In [ ]:
location = {'Neighborhood Latitude':  latitude, 'Neighborhood Longitude': longitude}
df_rent = df_rent.assign(**location)

In [ ]:
list_unknown_index = df_rent.index[df_rent['Neighborhood Latitude'] == -1].tolist()
print ("List of unknown Neighborhoods idexes : ", list_unknown_index)

df_unknown_names = df_rent.iloc[list_unknown_index, 0].reset_index()
list_unknown_names = df_unknown_names.iloc[:,1].tolist()
print ("List of unknown Neighborhoods names  : ", list_unknown_names)

# [27, 42, 43, 50, 51, 55]
#['Hunters Point', 'Northwestern Brooklyn', 'Northwestern Queens', 
#'Southwestern Queens', 'Stuyvesant Town - Peter Cooper Village', 'Theater District']
df_rent.head()

Our user_agent <em>ny_explorer</em> is reused as shown below to get localisation coordinates for the neighborhoods contained in rental data.
After localisation, we found 7 references in rental neighborhoods without localisation coordinates.

Analyzing the results of rental data, it seems that some names represent sectors of NYC and covers several neighborhoods.

Let's find this missing data.

In [ ]:
address_NYC = ', New York City, NY'
geo_agent = "ny_explorer"
geolocator = Nominatim(user_agent=geo_agent)

def get_coords(ngh_list, df):
    latitude = []
    longitude = []
    unknown = 0
    # 'Neighborhood Latitude'  'Neighborhood Longitude'
    n_name = ', ' + ngh_list[0]
    #print (n_name, '  -> ',  ngh_list[1])
    
    for address in ngh_list[1] :
        #time.sleep(0.300)
        #print ("Address is : ", address)
        try:
            location = geolocator.geocode(address + n_name + address_NYC)
            lat = location.latitude
            lng = location.longitude
        except:
            lat = -1
            lng = -1
            #print('  -> Cannot load NYC coordinate address!')
            unknown += 1
            pass
    
        latitude.append(lat)
        longitude.append(lng)
        #print('   -> The geograpical coordinate of {} are : {}, {}.'.format(address, lat, lng))
    
    print ('Unknown = ', unknown)
    BR1_avg = df.ix[ngh_list[2],1]
    budget = df.ix[2,2]
    list_coords = []
    
    for i, name in enumerate(ngh_list[1]):
        print(i, name, BR1_avg, budget, latitude[i], longitude[i])
        list_coords.append([name, BR1_avg, budget, latitude[i], longitude[i]])

    df_ngh = pd.DataFrame([item for item in list_coords])
    df_ngh.columns = df.columns
    df = df.append(df_ngh, ignore_index=True)
    
    return unknown, df

To find the missing data, I had to :
 - use Web https://www.latlong.net/convert-address-to-lat-long.html for manually retrieve coordinates for 4 of these neighborhoods.
 - translate sectors of neighborhoods by the corresponding list of official neighborhoods and again retrieve coordinates for them.

In [ ]:
#Neighborhood 	1BR 	Budget 	Neighborhood Latitude 	Neighborhood Longitude
# Missing
#[       27,                           42,                         43, 
#       50,                                51,                         55]
#['Hunters Point' : done, 'Northwestern Brooklyn', 'Northwestern Queens', 
#'Southwestern Queens', 'Stuyvesant Town - Peter Cooper Village', 'Theater District']

# These 4 coordinates have been manually retrieved via the 
# Web https://www.latlong.net/convert-address-to-lat-long.html
Hunters_Point = [40.718804, -73.805154]       # 27 -> 'Queens',
df_rent.ix[27, 3] = Hunters_Point[0]
df_rent.ix[27, 4] = Hunters_Point[1]

Peter_Cooper_Village =[40.734043, -73.977623] # 51 Northern Brooklyn Bedford-Stuyvesant 
df_rent.ix[51, 3] = Peter_Cooper_Village[0]
df_rent.ix[51, 4] = Peter_Cooper_Village[1]

Theater_District = [40.760213, -73.980105]    # 55 Midtown Manhattan
df_rent.ix[55, 3] = Theater_District[0]
df_rent.ix[55, 4] = Theater_District[1]

# Try to retrieve these ones below via geocoder
Northwestern_Brooklyn = ['Brooklyn',
                       ['Brooklyn Heights','Brooklyn Navy Yard','Clinton Hill',
                        'DUMBO', 'Downtown Brooklyn','Fort Greene','Prospect Heights',
                        'Vinegar Hill'
                       ],
                      42]

Northwestern_Queens = ['Queens',
                       ['Corona','East Elmhurst','Glendale','Jackson Heights',
                        'Maspeth','Middle Village','Ridgewood'],
                      43]

Southwestern_Queens = ['Queens',
                       ['Forest Park', 'Howard Beach', 'Ozone Park', 'Richmond Hill',
                        'South Ozone Park', 'Woodhaven'],
                      50]

print ('before size rent = ', df_rent.shape)
df_rent_plus = df_rent.copy()

u, df_rent_plus = get_coords(Northwestern_Brooklyn, df_rent_plus)
u1, df_rent_plus = get_coords(Northwestern_Queens, df_rent_plus)
u2, df_rent_plus = get_coords(Southwestern_Queens, df_rent_plus)

print ('Unknown = ', u+u1+u2, ' -> after size rent = ', df_rent_plus.shape)

df_rent_plus

In [ ]:
# Complete unknown location (get what we can from internet 'https://latitude.to')
# - from renthp : get list of associated neighborhoods
# - with geocoder retrieve associated locations
# - then format and add these items to renthop dataframe
# Last Remove Unknown #[27, 39, 42, 43, 50, 51, 55, 65]  [27, 42, 43, 50, 51, 55, 59] 

# Clean dataframe
list_u_index = df_rent_plus.index[df_rent_plus['Neighborhood Latitude'] == -1].tolist()
df_rent_plus = df_rent_plus.drop(list_u_index)
df_rent_plus.reset_index(drop=True, inplace=True)

print ('Unknown = ', list_u_index, ' -> after cleaning rent_plus size = ', df_rent_plus.shape)

In [ ]:
list_u_index = df_rent_plus.index[df_rent_plus['Neighborhood Latitude'] == -1].tolist()
list_u_index

Now missing data has been completed, let's clean the duplicates.

In [ ]:
print ('Number of neighborhood name duplicates =', 
      len(df_rent_plus['Neighborhood']) - len(df_rent_plus['Neighborhood'].unique()))

So, we will need to remove the duplicated rows be sure that each neighborhood is uniquely defined !

There are 4 neighborhoods names duplicated as seen above. Let's drop duplicated rows and reset index. See below 

In [ ]:
df_rent_plus.drop_duplicates(['Neighborhood'], inplace=True)
df_rent_plus.reset_index(drop=True, inplace=True)
print ('Number of neighborhood name duplicated after cleaning =', 
      len(df_rent_plus['Neighborhood']) - len(df_rent_plus['Neighborhood'].unique()),
      ' new size = ', df_rent_plus.shape )

In [ ]:
df_rent_plus.Budget = df_rent_plus['1BR'].apply(lambda x: rescaleBudgetLabel(x))
df_rent_plus.Budget = set_col_cat(df_rent_plus.Budget)

filename = 'NYC_Renthop_Neighborhoods_with_coords_cleaned.csv'
if not_present(filename):
    df_rent_plus.to_csv('NYC_Renthop_Neighborhoods_with_coords_cleaned.csv')
    print ('Created Clean Data file : NYC_Renthop_Neighborhoods_with_coords_cleaned.csv')

Ok, now we have a clean data set of neighborhoods with average rentals. 

To finish the data processing part, we need to merge the last dataframe containing the venues with walk scores.

In [ ]:
df_rent_final =  pd.read_csv('NYC_Renthop_Neighborhoods_with_coords_cleaned.csv')
df_rent_final.drop(df_rent_final.columns[[0]], axis=1, inplace=True)
df_rent_final.Budget = df_rent_final['1BR'].apply(lambda x: rescaleBudgetLabel(x))
df_rent_final.Budget = set_col_cat(df_rent_final.Budget)
print(df_rent_final.dtypes, df_rent_final.shape)
df_rent_final.sort_values(['1BR','Budget'],ascending=[False,False])
# df_rent_final.to_csv('NYC_Renthop_Neighborhoods_with_coords_cleaned.csv')

In [ ]:
merged_neighborhood = pd.merge(df_venues_cleaned_wscores, df_rent_final, on=['Neighborhood'], how='inner')
merged_neighborhood

In [ ]:
len(merged_neighborhood['Neighborhood'].unique())

Only 47 neighborhoods have matched ! 

**Because the names  are not exactly identical, venues for 38 neighborhoods have been lost so that we can only work on 2 on 11279 !**

**I decided to process again the data but this time starting from the renthop file to retrieve the venues et perform the same work as previously done to obtain clean data ready for analysis.**

In [ ]:
import json
import time

target_root_categories = ['4d4b7105d754a06374d81259','4d4b7104d754a06370d81259',
                          '4d4b7105d754a06373d81259','4d4b7105d754a06376d81259',
                          '4d4b7105d754a06377d81259','4d4b7105d754a06378d81259'
                         ]

# define function to format venues address
# Lot of problems with missing fields ex: 'address' or 'city'
# -> so take formattedAddress
def get_address(flocation):
    # print ('location(address) = ', location['formattedAddress']) #['address'], location['city'])
    address = flocation[0] + ', ' + flocation[1]
    return address

# build the list of chosen categories (easier to find a job)
root_categories = target_root_categories[0]
for cat in  target_root_categories[1:]:
    root_categories = root_categories + ',' + cat

# build the dataframe of selected categories of venues
# Normally, we are limited by the API to 50 retrieved venues
def getNearbyFilteredVenues(names, latitudes, longitudes,
                            br1s, budgets,
                            categories="", radius=500, limit=100):
    # Use search API with the list of selected root categories
    url_str = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&intent=browse&categoryId={}&ll={},{}&radius={}&limit={}"

    # Retrieve list of target venues in a radius of 500 m from the coordinates
    # of each of the 306 identified neighborhoods and complying the given categories
    venues_list=[]
    i = 0
    for name, lat, lng, br1, budget in zip(names, latitudes, longitudes, br1s, budgets):
        #print(i, " -> ", name)
        time.sleep(0.1)
        url = url_str.format(CLIENT_ID, CLIENT_SECRET, VERSION,
                             categories,lat, lng, 
                             radius, 50)
        try:
            results = requests.get(url).json()["response"]["venues"]
        except:
            print(i, " -> ", name, " -> No venues !!!!")
            continue
            pass

        print(i, " -> ", name, " -> venues number = ", len(results))

        # For each resulting venue list for this neighborhood
        # build the associated data : name address coordinates etc. 
        for v in results:
            #parsed_venue = json.loads(venue)
            #print("->",json.dumps(v, indent=4, sort_keys=True))
            #
            venues_list.append([(
                name,                       # neighborhood name
                lat,                        # neighborhood latitude
                lng,                        # neighborhood longitude
                br1,                        # Avg 1BR price in neighborhood
                budget,                     # Avg price ranking
                v['name'],                  # venue name
                get_address(v['location']['formattedAddress']), # venue address
                v['location']['lat'],       # venue latitude
                v['location']['lng'],       # venue longitude
                v['categories'][0]['name']  # venue category
            )])
    
        i = i +1

    # Once we have registered all retrieved venues, put them in a new filtered dataframe
    nearby_filtered_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_filtered_venues.columns = ['Neighborhood',
                                      'Neighborhood Latitude',
                                      'Neighborhood Longitude',
                                      '1BR', 'Budget',
                                      'Venue',
                                      'Address',
                                      'Venue Latitude',
                                      'Venue Longitude',
                                      'Venue Category']

    return nearby_filtered_venues

print("Filtered Venues Functions defined !")

In [ ]:
# Try to load from local file system in case we did this before
import os
cwd = os.getcwd()
print ('cwd =', cwd)

fromFile = False
try:
    newyork_renthop_filtered_venues = pd.read_csv('./NYC_Renthop_Neighborhoods_filtered_venues.csv')
    newyork_renthop_filtered_venues.drop(newyork_renthop_filtered_venues.columns[[0]], axis=1, inplace=True)
    newyork_renthop_filtered_venues.Budget = set_col_cat(newyork_renthop_filtered_venues.Budget)
    print('NYC Renthop Neighborhoods Filtered Venues data loaded.')
    fromFile = True
except:
    pass

#Neighborhood 	1BR 	Budget 	Neighborhood Latitude 	Neighborhood Longitude
# If not already loaded use the Foursquare API to get the data 
if not fromFile:
    print ('Data file for filtered venues not existing ! Call API')
    newyork_renthop_filtered_venues = getNearbyFilteredVenues(names=df_rent_final['Neighborhood'],
                                                              latitudes=df_rent_final['Neighborhood Latitude'],
                                                              longitudes=df_rent_final['Neighborhood Longitude'],
                                                              br1s=df_rent_final['1BR'],
                                                              budgets=df_rent_final['Budget'],
                                                              categories=root_categories,
                                                              radius=500,
                                                              limit=100
                                                              )
    # Export the dataframe values to a local file  
    newyork_renthop_filtered_venues.to_csv('NYC_Renthop_Neighborhoods_filtered_venues.csv')

newyork_renthop_filtered_venues.head()

In [ ]:
print ('Number of retrieved filtered venues is : ', newyork_renthop_filtered_venues.shape[0])
newyork_renthop_filtered_venues.head()

### Retrieve Walk Scores

Following the format given above, thanks to the function below, I built the url from the parameters identifying a venue.    

In [ ]:
def get_url(address, lat, lon, wsapikey=""):
    url = 'http://api.walkscore.com/score?format=json&address={}&lat={}&lon={}&transit=1&bike=1&wsapikey={}'.format(
            address, 
            lat,
            lon,
            wsapikey)
    return url

### Walk Score Retrieval Functions Definition

The functions below retrieve the walk scores and build the new dataframe with the added scores for each venue.

In [ ]:
# get the score of venues.
# Normally, we are limited by the API to 5000 calls per day
# An example of the results from WS API that we have to process
result_str = '{\
    "bike": {\
        "description": "Bikeable",\
        "score": 54\
    },\
    "description": "Somewhat Walkable",\
    "status": 1,\
    "walkscore": 59\
}'


def getScore(v_name, lat, lon, adr):
    url = get_url(adr, lat, lon, wsapikey=WSAPIKEY)
    scores_descs = [-1, "", -1, "", -1, ""]
    results = requests.get(url).json() # json.loads(result_str) #  json.loads(result_str) # 
    status = results['status']
    
    bscore = tscore = -1
    bdescr = tdescr = ""
    if (status != 1):
        print("Venue :", v_name, " -> Error status returned = ", status)
    else :
        #print("->",json.dumps(results, indent=4, sort_keys=True))
        wscore, wdescr = results['walkscore'],results['description']
        try:
            bscore, bdescr = results['bike']['score'], results['bike']['description']
        except:
            pass
        try:
            tscore, tdescr = results['transit']['score'], results['transit']['description']
        except:
            pass

        scores_descs = [wscore, wdescr, bscore, bdescr, tscore, tdescr]

    return scores_descs

# Neighborhood Latitude 	Neighborhood Longitude 	Venue 	Venue Latitude 	Venue Longitude 	Venue Category
def getWalkScoreVenue(venues_score_df,beg=0):
    # Retrieve list of target venues in a radius of 500 m from the coordinates
    # of each of the identified neighborhoods and compute walk score
    
    venues_df_ncol = len(venues_score_df.columns)
    for index, row in venues_score_df.iterrows():
        name = row['Venue']
        lat = row['Venue Latitude']
        lng = row['Venue Longitude']
        adr = row['Address']
        
        #time.sleep(0.001)
        results = getScore(name, lat, lng, adr)
       
        i = 0
        #r_num = index - beg
        venues_score_df.loc[index,'Walk_score'] = results[i] # wscore
        i += 1
        venues_score_df.loc[index,'Walk_descr'] = results[i] # wsdescr
        #print('index = ', index, venues_score_df.loc[[index]])
        if (index % 100) == 0:
            print('index = ', index, venues_score_df.loc[index,'Walk_descr'], venues_score_df.loc[index,'Walk_score'])
    
    return venues_score_df

# Try to load from local file system in case we did this before
def load_scores(df_venues, beg, end):
    filename = 'NYC_venues_walkability_{}_{}.csv'.format(beg,end)
    fromFile = False
    try:
        df_venues = pd.read_csv(filename)
        df_venues.drop(df.columns[[0]], axis=1, inplace=True)
        df_venues.Budget = set_col_cat(df_venues.Budget)
        df_venues.Walk_descr = set_col_cat(df_venues.Walk_descr, ws_cat)
        print('NYC Neighborhoods Venues Walkability Scores data loaded from file :', 
              filename)
        
        fromFile = True
    except:
        pass

    # If not already loaded : use the Foursquare API to get the data
    if not fromFile:
        print (filename, ' : Data file for filtered venues not existing ! Call API')
        # Add scores columns for walkability and bike
        df_venues = df_venues.assign(Walk_score=-1, Walk_descr=None)
        df_venues = getWalkScoreVenue(df_venues,beg)
        df_venues.Budget = set_col_cat(df_venues.Budget)
        df_venues.Walk_descr = set_col_cat(df_venues.Walk_descr, ws_cat)

        # Export the dataframe values to a local file  
        #print(df)
        #df_venues.to_csv(filename)
        print ('Venues with Walk Scores Data file created : ', filename)
    return df_venues

In [ ]:
WSAPIKEY = cred_df.iat[3, 0]
#df_0_3 = pd.DataFrame()
df_0_3 = newyork_renthop_filtered_venues[0:3].copy()
df_0_3 = load_scores(df_0_3, 0, 3)
df_0_3.dtypes
df_0_3

### Data Reworking 
Looking at the budget with 3 values, sometimes a 1BR amount is labelled for example 'Pricey' while another higher amount is labelled 'Average'. It is not very consistent.

It is why, I think it is more relevant to rescale this column by associating a more understandable label to the price '1BR' value.

So we will rework the Budget column so that new categories will remap on this table :  

|  Label    |    Inf    |  Sup  |
|:---------:|:---------:|:-----:|
| *Cheap*   |     0     | 2149  |
| *Average* |    2150   | 3149  |
| *Pricey*  |    3150   |  '>'  |

So, let's compute and associate a categorical label to the registered Budget value.   

In [ ]:
#WSAPIKEY = cred_df.iat[3, 0]
df_0_3513 = pd.DataFrame()
df_0_3513 = load_scores(newyork_renthop_filtered_venues[0:].copy(), 0, 3513)
df_0_3513.drop(df_0_3513.columns[[0]], axis=1, inplace=True)
df_0_3513['Walk_descr']= df_0_3513['Walk_score'].apply(lambda x: getWalkLabel(x))
df_0_3513.Walk_descr = set_col_cat(df_0_3513.Walk_descr, ws_cat)

df_0_3513.Budget = df_0_3513['1BR'].apply(lambda x: rescaleBudgetLabel(x))
df_0_3513.Budget = set_col_cat(df_0_3513.Budget)

In [ ]:
#df_0_3513.to_csv('NYC_venues_walkability_0_3513.csv')
df_venues.sort_values(['1BR','Budget'],ascending=[False,False])

In [ ]:
# Clean dataframe
list_u_index = df_0_3513.index[df_0_3513['Walk_score'] == -1].tolist()
print(list_u_index)
if (len(list_u_index) > 0):
    df_0_3513 = df_0_3513.drop(list_u_index)
    df_0_3513.reset_index(drop=True, inplace=True)

### End of Data Processing 

Normally at this step, all venues have been collected and processed : data cleaning, missing data etc...
For convenience reasons, this data will be stored in a file that can be reloaded in a dataframe at any time for future operations.

In [ ]:
filename = 'NYC_venues_walkability_{}_{}.csv'.format(0,3513) #NYC_venues_walkability_0_3513.csv
df_venues = pd.read_csv(filename)
df_venues.drop(df_venues.columns[[0]], axis=1, inplace=True)
df_venues['Walk_descr']= df_venues['Walk_score'].apply(lambda x: getWalkLabel(x))
df_venues['Budget']= df_venues['1BR'].apply(lambda x: rescaleBudgetLabel(x))
df_venues.Budget = set_col_cat(df_venues.Budget)
df_venues.Walk_descr = set_col_cat(df_venues.Walk_descr, ws_cat)

print('NYC Neighborhoods Venues Walkability Scores data loaded from file :', filename)
print(df_venues.dtypes, '\n',' -> shape = ', df_venues.shape)
df_venues

Now we can summarize what we know about the new dataframe with the added scores for each venue.

In [ ]:
print ('Data Summary\n------------\n')
print('Total number of {:<26} = {} '.format('venues', df_venues.shape[0]))
print('Total number of {:<26} = {} '.format('venue attributes', df_venues.shape[1]))
print('Total number of {:<26} = {} '.format('represented neighborhoods', len(df_venues['Neighborhood'].unique())))
print('Total number of {:<26} = {} '.format('unique venue categories', len(df_venues['Venue Category'].unique())))

It looks good and that ends the data gathering and processing phases.

Hard work and a lot of trials have been necessary to retrieve with APIs and sometimes manually the full information needed to our analysis.

#### Conclusion : data retrieval and processing

So now we have all the venues in the retrieved set of 3508 samples representing 81 neighborhoods in New York area with walk scores and rental data.

In the present study, due to lack of time, we will only consider walk score, not bike or transit scores that are also useful information needed to refine the results we would like to present to the interested stakeholders.

We also could extend the information to all 305 neighborhoods of NYC but it needed more work to retrieve for example exhaustive rental information.

Idem for the number of venues limited for our profile to 50 per search call and walkability scores limited to 5000 per day that is clearly not sufficient in time to cover all potential venues existing in all neighborhoods of NYC.  

The strategy concerning missing data, duplicates could also be refined : dropping is not always the more efficient action ! We tried also to calculate approximate values but from my point of view, the results are not entirely satisfactory.  Developing more elaborated algorithms needs more time I was not able to dedicate to this study.

This concludes our data gathering phase. 

Let's continue to the next step : how this data will be use for analysis leading to find the best places in the 81 neighborhoods of NYC for a student or person wanting to rent a 1BR apartment in a neighborhood with potential work places easily accessible on foot!

# Methodology

In this project we will direct our efforts on detecting areas of New York City that have the highest density of venues located in the highest walk-scored places with the lowerest rental rates.

As said in the _**Data**_ section , we will limit our analysis to the ares containes in the 81 neighborhoods of NYC having information about walkability scores and rental rates.

Below, I detail the methodology I used to carry out this project.

- Step 1:

    Previously, in a we have already collected the amount of **data: location, category, walk score and average rent for each potential workplace located in 81 neighborhoods of New York City. Each potential workplace have been identified and classified according to Foursquare categorization**.


- Step 2:

    The analysis will be based on calculation and exploration of '**venues**' across the different neighborhoods of NYC. We will use **heatmaps** to identify a few promising areas with highest density of venues. walkscores and the lowest rental values and focus our attention on those areas.


- Step 3:

    In this final step we will focus on most promising areas and within those create **clusters of locations that meet the requirements** established at the beginning of the project defined with stakeholders.
    
    The locations targeted are those with the highest '**walk scores**' and the '**lowest rates of rent**'. These areas will be classified according to the density of venues in the  vicinity having the highest rate of **pedestrian friendliness in a radius of 500 meters**,  and we want locations **with the lowest rates of rent in radius of 500 meters**.
    
    A map will be presented to the stakeholders, displaying all such locations grouped in createed clusters (using **k-means clustering**) of those locations to identify these specific target areas in neighborhoods which constitute a starting directory of adresses to be explored and searched by the astakeholders identified as the best places to choose the cheapest home and find work in the nearest vicinity.

# Analysis <a name="analysis"></a>

##### Let's perform some basic explanatory data analysis and derive some additional info from our raw data. 

3508 retrieved venues that are completely filled by rental data !
Let's see the number of venues grouped by neighborhood.

In [ ]:
df = df_venues.groupby('Neighborhood', as_index=False).count() #
df = df.sort_values('Venue',ascending=False).reset_index(drop=True)
df['Venues CumSum'] = df['Venue'].cumsum()
df[['Neighborhood','Venue', 'Venues CumSum']]

Now we know that the first 53 elements in range [0, 52] (-> Garment District) have all 50 retrieved venues and cumulate from 0 to 2650 venues on a total of 3508 for all 81 venues.

Neighborhoods in last range [54, 80]  (Battery Park City -> Forest Park) cumulate from 2651 to 3508 venues

Unfortunately we have no information on real repartition of venues because the most part of neighborhoods that cumulate more than 50 retrieved venues are limited to 50 cause API limitation.

So let's try some visualization on the retrieved data.

Let's now compute the walkability for these neighborhoods in this order.

- loop on neighborhoods of NYC
   - loop on each venue
     - compute walkability 
     - add walk '(bike/transit) score values in new columns of newyork_filtered_venues
   - compute mean values by neighborhood if needed
  
Discussion :
 - we can perform the call for each of the 11311 retrieved venues (need at least 3 days)
 - we can just perform the walkability calculation on the first 5000 venues but 143 neighborhoods have 50 venues
 - we can pick a number of venues according to the density of retrieved venues for each neighborhood so that
   all neighborhoods are covered and can be compared in term of walkability

In [ ]:
df = df_venues.copy()
df = df.groupby('Neighborhood', as_index=False).count().sort_values('Venue',ascending=True).reset_index(drop=True)#
#df = df.sort_values('Venue',ascending=False).reset_index(drop=True)
df['Venues CumSum'] = df['Venue'].cumsum()
df['Number Venues'] = df['Venue']
del df['Venue'] # , 'Address', 'Venue Latitude' , 'Venue Longitude' , 'Venue Category' , 'Walk_score' , 'Walk_descr']
lst = ['Address', 'Venue Latitude' , 'Venue Longitude' , 'Venue Category' , 'Walk_score' , 'Walk_descr']
for item in lst:
    del df[item]
df = df.sort_values('Neighborhood',ascending=True).reset_index(drop=True)
df.head()

### Adding Categorical Information at Neighborhood level : Mean Walk Score

###### How Walk Score Works

Walk Score helps you find a walkable place to live. Walk Score is a number between 0 and 100 that measures the walkability of any address.

The sWalk score in a number indicating a category of place as described below :

- 90–100	: **[Walker’s Paradise]**
              Daily errands do not require a car
- 70–89	    : **[Very Walkable]**
              Most errands can be accomplished on foot
- 50–69	    : **[Somewhat Walkable]**
              Some errands can be accomplished on foot
- 25–49	    : **[Car-Dependent]**
              Most errands require a car
- 0–24	    : **[Car-Dependent]**
              Almost all errands require a car

Let's compute and associate a categorical label to the registered neighborhoods relative to the walk score of each retrieved venues.  

In [ ]:
df1= df_venues.copy()
#df['WS_Mean']= df.groupby('Neighborhood')['Walk_score'].transform('mean')#
df_ws_mean = df1.groupby('Neighborhood').agg('mean').reset_index()
df_ws_mean = df_ws_mean.sort_values('Neighborhood',ascending=True)
df_ws_mean = df_ws_mean[['Neighborhood', 'Walk_score']]
df_ws_mean[['Number Venues']]= df[['Number Venues']]
df_ws_mean.rename(index=str, columns={"Walk_score": 'WS_mean'}, inplace=True)
df_ws_mean.WS_mean = df_ws_mean.WS_mean.round().astype(int)
df_ws_mean['WS_descr']= df_ws_mean['WS_mean'].apply(lambda x: getWalkLabel(x))
df_ws_mean.WS_descr = set_col_cat(df_ws_mean.WS_descr, ws_cat)
df_ws_mean

In [ ]:
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

##### Let's build or reload raw data needed for visualization and basic  explanatory data analysis. 

In [ ]:
def build_info_neighborhood():
    df_rent_final =  pd.read_csv('NYC_Renthop_Neighborhoods_with_coords_cleaned.csv')
    df_rent_final.drop(df_rent_final.columns[[0]], axis=1, inplace=True)
    df_rent_final.Budget = set_col_cat(df_rent_final.Budget)
    #print(df_rent_final.dtypes)

    filename = 'NYC_Information_Viz.csv'
    if not not_present(filename):
        df_info_viz = pd.read_csv('NYC_Information_Viz.csv')
        df_info_viz.drop(df_info_viz.columns[[0]], axis=1, inplace=True)
        df_info_viz.reset_index(drop=True)
        df_info_viz.Budget = set_col_cat(df_info_viz.Budget, budget_cat)
        df_info_viz.WS_descr = set_col_cat(df_info_viz.WS_descr, ws_cat)
        print('Data ready for visualisation reloaded from : ', filename)
    else :
        print(filename, ' not existing ! Data for visualisation to be rebuild from previous steps !')
        df_info_viz = df_rent_final.copy().sort_values('Neighborhood',ascending=True)
        df_ws_mean, df_info_viz = [d.reset_index(drop=True) for d in (df_ws_mean, df_info_viz)]
        df_ws_mean.index =  df_info_viz.index
        df_info_viz = df_info_viz.join(df_ws_mean[['Number Venues', 'WS_mean', 'WS_descr']]).reset_index(drop = True)
        df_info_viz.to_csv('NYC_Information_Viz.csv')

    # or pd.concat([df_info_viz, df_ws_mean[['Number Venues', 'WS_Mean']]], axis=1)
    df_info_viz.sort_values(['WS_descr', 'Budget', 'WS_mean'], ascending=[False,True,False], axis=0, inplace=True)
    
    return df_info_viz
    
df_info_viz = build_info_neighborhood()
print(df_info_viz.dtypes, len(df_info_viz.index))
df_info_viz.head(2)

## 1. Graphical Data Analysis 

In [ ]:
df_info_viz.sort_values(['Number Venues'], ascending=False, axis=0, inplace=True)
count, bin_edges = np.histogram(range(df_info_viz.shape[0]), bins =8)
print(count) # frequency count
print(bin_edges) # bin ranges, default = 10 bins

In [ ]:
df_info_viz['Index'] = range(df_info_viz.shape[0])
df_info_viz.head()


###  a. Venue Density 

In [ ]:
df_info_viz['Number Venues'].plot(kind='hist', figsize=(8, 6))
plt.title('Histogram of number of venues retrieved in NYC Neighborhoods') # add a title to the histogram
plt.ylabel('Number of Neighborhoods') # add y-label
plt.xlabel('Number Venues') # add x-label

plt.show()

In [ ]:
df_t = df_info_viz.copy()
df_t.set_index(['Index'], inplace=True)
df_t = df_t.loc[df_t.index, 'Number Venues']
df_t.head()

In [ ]:
# generate histogram
df_t.plot(kind='bar', figsize=(10, 6))

plt.title('Histogram of number of Venues by neighborhood in NYC')
plt.ylabel('Number of venues')
plt.xlabel('Neighborhood Index')

plt.show()

In [ ]:
df_t = df_info_viz.copy()
#df_t.set_index(['Neighborhood'], inplace=True)
df_t.sort_values(['1BR'], ascending=False, axis=0, inplace=True)
df_t.reset_index(drop=True, inplace=True)

###  b. Rental Prices in Neighborhoods 

In [ ]:
# generate Bar
count, bin_edges = np.histogram(df_t['1BR'])
df_t['1BR'].plot(kind='hist', figsize=(8, 5))

plt.title('Histogram of Average 1BR Rental by neighborhood in NYC')
plt.ylabel('Number of neighborhoods')
plt.xlabel('Average 1BR Rental')

plt.show()

In [ ]:
# generate Bar
count, bin_edges = np.histogram(df_t['1BR'])
df_t['1BR'].plot(kind='bar', figsize=(8, 5))

plt.title('Histogram of Average 1BR Rental by neighborhood in NYC')
plt.ylabel('Average 1BR Rental')
plt.xlabel('Neighborhood Index')

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

In [ ]:
df_iv= df_info_viz[['Neighborhood','1BR','Budget']].copy()
df_iv['1BR'] = df_iv['1BR'].astype(int)
df_iv.sort_values(['1BR'], ascending=False, axis=0, inplace=True)
df_iv.head()

In [ ]:
df_br = df_iv.copy()
df_br.head()

###  c. Statistics Summary 

Let's go further in our explanatory data analysis and see if some additional information from our raw data is needed. 

First let's calculate some basics statistics on rentals over the neighborhoods. That could be useful information foor the stakeholders.

####  1. Rental Prices Description  

In [ ]:
#Describe summary statistics on 1BR over the neighborhoods
print (df_info_viz['1BR'].describe())

(df_info_viz['1BR']).plot(kind='box', figsize=(8, 6))

plt.title('Box plot of Average Rentals of 1 Bedroom apartment')
plt.ylabel('1BR Avg price')

plt.show()

- Minimum: Smallest number in the dataset                            : min  = 1625
- First quartile: Middle number between the minimum and the median   : 25%  = 2250
- Second quartile (Median): Middle number of the (sorted) dataset    : 50%  = 2950
- Third quartile: Middle number between median and maximum           : 75%  = 3312
- Maximum: Highest number in the dataset                             : max  = 4391

It has been counted on 81 values and the mean Average Rental values is around $2798.

That seems to be a quite high amount of money for a student ! 

We could also see how the neighborhood are considered relative to the 'Budget' value, a category indicating the observed rental price compared to what it should be. 

####  2. Rental Price Percentages Categories Description  

In [ ]:
df_bb = df_iv.copy()
df_bbg  = df_bb.groupby('Budget', as_index=False).count().sort_values('Budget',ascending=True).reset_index(drop=True) #
df_bbg = df_bbg[['Budget', 'Neighborhood']]
total_n = df_bbg['Neighborhood'].sum()
df_bbg['Perc%'] = ((df_bbg[['Neighborhood']] / total_n) * 100).round()
print ('Percentage of Neighborhoods per Price category')
df_bbg[['Budget','Perc%']]

In [ ]:
print (df_bbg['Perc%'].describe())

df_bbg['Perc%'].plot(kind='box', figsize=(8, 6))

plt.title('Box plot of Neighborhoods Percentages by Budget category')
plt.ylabel('Percentage of Neighborhoods')

plt.show()

####  3. Rental Price Repartition by  Categories Description  

In [ ]:
df_bb = df_iv.copy()
df_bb['1BR'] = (df_info_viz['1BR']).astype(int)
df_bb = df_bb.sort_values(['Budget', '1BR'] ,ascending=[True,True])
tips = sns.load_dataset("tips")
ax = sns.catplot(x="Budget", y="1BR", data=df_bb)

Let's explore now data for Average rental by Walkability scores.

###  d. Walkability and Rental Prices Relationship  

In [ ]:
#ws_cat = ['Car-Dependant', 'Somewhat Walkable', 'Very Walkable', 'Walker’s Paradise']
#budget_cat =['Cheap', 'Average', 'Pricey']
dd = df_info_viz[['Budget', 'WS_descr', 'Number Venues']].copy()
df_11 = pd.DataFrame([], index=budget_cat, columns=ws_cat)
df_11.reset_index(drop=True)
df_11

####  1. Neighborhoods by Walkability Scores and Rental Price Categories  

In [ ]:
dfd_ngh = df_11.copy()
dd = df_info_viz[['Budget', 'WS_descr', 'Number Venues']].copy()
dfd_ngh.apply(calc_total, args =('Count',dd), axis=1)
df_nac =  dfd_ngh.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]
total_n = dfd_ngh.astype(float).sum().sum().astype(int)
print ('Total Neighborhoods = ', total_n)
total_ac_vp = df_nac.astype(float).sum().astype(int)
print('Total number of neighborhoods with lowest Rentals and more than Very Walkable = ', total_ac_vp.sum(),
      '\n -> Representing a rate of : ', str(round(total_ac_vp.sum()/total_n * 100).astype(int))+'%')
print()
dfd_ngh 

In [ ]:
dfd_ngh_perc = dfd_ngh.copy()
df_nac_perc =  dfd_ngh_perc.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]
total_n = dfd_ngh_perc.astype(float).sum().sum().astype(int)
total_ac_vp_perc = df_nac_perc.astype(float).sum().sum().astype(int)
print ('Total Neighborhoods = ', total_n)
print ('Total affordable Neighborhoods in more than Very Walkable areas = ', total_ac_vp_perc)
print('Total Percentage of affordable Neighborhoods in top 2 walkable areas = ', 
      str(round(total_ac_vp_perc.sum()/total_n * 100).astype(int)) +
      '%')
dfd_ngh_perc= dfd_ngh_perc.divide(total_n).mul(100).astype(float).round(0)
print('\n-------------------------------------------------------------------\n')
print('Table of Percentages of Neighborhood Walkability Scores by Areas Prices')
dfd_ngh_perc

Well, we can see clearly that more than half (45 on 81) i.e 56% of all neighborhoods prices are relatively affordable. 

All 'Pricey' neighborhoods (33 on 81) are highly walkable and represent about 41% of the total.

Let's repeat the same calculus to explore now the number of venues for potential workplaces relative to Walkability scores.

####  2. Venues Targeted by Walkability Scores and Rental Price Categories  

In [ ]:
dfd_nv = df_11.copy()

dfd_nv.apply(calc_total, args=('Number Venues', df_info_viz[['Budget', 'WS_descr', 'Number Venues']].copy()), axis=1)
df_ac =  dfd_nv.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]

total_v= dfd_nv.loc[['Cheap','Average','Pricey'],['Very Walkable','Walker’s Paradise']].astype(float).sum().sum().astype(int)
print ('Total Venues Very Walkable or Walker\'s Paradise = ', total_v)
total_ac_vp = df_ac.astype(float).sum().astype(int)
print('Total number of venues with lowest Rentals and more than Very Walkable = ', total_ac_vp.sum(),
      '\n -> Representing a rate of : ', str(round(total_ac_vp.sum()/total_v * 100).astype(int))+'%')
#dfd_nv
dfd_nv.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]

In [ ]:
#def numpy_numexpr_app(df):
#    a = df.values.astype(float)
#    N =  total_v
#    return np.around(pd.DataFrame(a/N, columns=df.columns, index=df.index),decimals=2)

#dfd_nv_p = numpy_numexpr_app(dfd_nv)
#dfd_nv_p.head(6)

def format_x(x):
    x =float(x)
    if not math.isnan(x):
        val = str(x).replace('.0', '%')
    else:
        val ='-'
    return val

dfd_nv_perc = dfd_nv.copy()

dfd_vac_perc =  dfd_nv_perc.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]
#total_v = dfd_nv_perc.astype(float).sum().sum().astype(int)

total_vac_vp_perc = dfd_vac_perc.astype(float).sum().sum().astype(int)
print ('Total Venues Very Walkable or Walker\'s Paradise = ', total_v)
print ('Total affordable Venues in more than Very Walkable areas = ', total_vac_vp_perc)
print('Total Percentage of Venues in top 2 walkable areas = ', 
      str(round(total_vac_vp_perc.astype(int)/total_v * 100).astype(int)) +
      '%')
dfd_nv_perc= dfd_nv_perc.divide(total_v).mul(100).astype(float).round(0)
print('\n-------------------------------------------------------------------\n')
print('Table of Percentages of Walkability Scores by Areas Prices')
dfd_nv_perc

In [ ]:
# As for the chart, use a figure size of (20, 8):
# - bar width of 0.8,
# - use color #5cb85c for the Very interested bars, 
#       color #5bc0de for the Somewhat interested bars, and 
#       color #d9534f for the Not interested bars,
# - use font size 14 for the bar labels, percentages, and legend,
# - use font size 16 for the title, and,
# - display the percentages above the bars as shown above, and 
#   remove the left, top, and right borders.
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt
from decimal import Decimal

mpl.style.use('ggplot') # optional: for ggplot-like style

w_vi = 'chartreuse'         # 'Walker’s Paradise'
v_ni = 'springgreen'        # 'Very Walkable'
s_si = 'olive'              # 'Somewhat Walkable'
c_ci = 'darkgreen'          # 'Car-Dependant'

colors= list([c_ci, s_si, v_ni, w_vi])

# create bar plot
ax = dfd_nv_perc.plot(kind='bar', width=0.8, color=colors, figsize=(20, 8))

# set title and legend
# add title to the plot
ax.set_title('Percentage of Walkability Scores by Areas Prices', fontsize=16, fontweight="bold") 
leg = ax.legend(frameon=1, prop=dict(size=14,weight='bold'))
leg.get_frame().set_linewidth(2.0)

for label in ax.xaxis.get_ticklabels():
    label.set_fontsize(16)
    label.set_weight('bold')

# Remove vertical axe
plt.gca().axes.get_yaxis().set_visible(False)

# Add bottom line
plt.axhline(0, linewidth = 5.0, color='grey')

for p in ax.patches:
#    strperc = '{:^15}'.format(str(p.get_height()).split('.',1)[1]+ '%') with 0.68
    strperc = '{:^15}'.format(str(int(p.get_height()))+ '%') # with 68
    ax.annotate(strperc, (p.get_x(), p.get_height() + 0.6), fontsize=15, fontweight='bold')

plt.tight_layout()
plt.show()

We can see that all areas are massively Wakable and the highest is walkability score, the highest is the rental price.

So, the calculated numbers for venues correlate quite well those for neighborhoods (not a surprise, the opposite would have been surprising of course !). About 53% of all venues are located in 'affordable' areas with high walkability scores.

As previously observed, all venues located in 'Pricey' areas are also highly walkable and represent about 46% of the total.

In [ ]:
df_info_viz.head()

In [ ]:
df_bb.sort_values(['1BR', 'Budget'] ,ascending=[True,True])

Ok let's see another plotting results !

In [ ]:
df_ws_viz = df_info_viz.copy()
df_ws_viz = df_ws_viz.sort_values(['Budget','1BR'] ,
                                  ascending=[False,True]).reset_index(drop=True)
df_ws_viz

####  3.  Rental Price Categories  by Neighborhood Walkability Score Mean 

Below is plotting observed Pricing relative to Walkability mean in the Neighborhood using **catplot** !

In [ ]:
df_ws_viz = df_info_viz.copy().sort_values('Neighborhood',ascending=True).reset_index(drop=True)

df_ws_viz['1BR'] = df_ws_viz['1BR'].astype(int)
df_ws_viz = df_ws_viz.sort_values(['Budget','1BR'] ,
                                  ascending=[True,True]).reset_index(drop=True)
tips = sns.load_dataset("tips")
ax = sns.catplot(x="Budget", y="WS_mean", data=df_ws_viz)

And below the same as above but using **boxplot** !

In [ ]:
tips = sns.load_dataset("tips")
#ax = sns.catplot(x="WS_descr", y="1BR", data=df_ws_viz)
plt.figure(figsize=(8,6))
ax = sns.boxplot(x="Budget", y="WS_mean", data=df_ws_viz)
#ax = sns.boxplot(x="WS_mean", y="Budget", data=df_ws_viz)

####  4.  Neighborhood Walkability Score Mean  by '1BR' Rental Prices 

Try a **boxplot** to vizualize Walk score by 1 Bedroom-homes prices !

Well we can easily observe that the most highly wakable places are also the most costly ! 

In [ ]:
tips = sns.load_dataset("tips")
#ax = sns.catplot(x="WS_descr", y="1BR", data=df_ws_viz)
plt.figure(figsize=(8,6))
ax = sns.boxplot(x="WS_descr", y="1BR", data=df_ws_viz)

In [ ]:
df_ws_viz.sort_values(['WS_mean','1BR', 'Neighborhood'], ascending=[False,True,True]).reset_index(drop=True)

In [ ]:
df_venues_sorted = df_venues.copy().sort_values(['Walk_score','1BR', 'Venue Latitude', 'Venue Longitude','Neighborhood'],
                                                        ascending=[False,True,True,True,True]).reset_index(drop=True)
df_venues_sorted.head()

####  5. Conclusions 

Interesting information for the future tenants ! 

- Unfortunately major part of the rents are 'Pricey' : 41 %
- 1BRs with a Cheap rent are not so many : 22 %
- Even in Average prices, only a little bit more than 1/3 can be found : 41 %

We already found that :
    Minimun: Smallest number in the dataset : min = 1625
    First quartile: Middle number between the minimum and the median : 25% = 2250
    Second quartile (Median): Middle number of the (sorted) dataset : 50% = 2950
    Third quartile: Middle number between median and maximum : 75% = 3312
    Maximum: Highest number in the dataset : max = 4391

It has been counted on 81 values and the mean Average Rental values is around 2798.

A good news : most part of the neighborhoods (78 on a total of 81) are localised in areas with high walk scores (Walker’s Paradise or Very Walkable) and 53 of these 78 have the maximum of venues (50) allowing better chance to find a job.

So we would concentrate on these areas to propose our adresses. In fact we would like to build a multi-dimentional table of areas where the best profile would be :
- Highest Wakability scores
- Highest number of Venues
- Lowest Rental prices


|Budget     |    Inf    |  Sup  |           
|-----------|:---------:|:-----:|
| *Cheap*   |     0     | 2149  |
| *Average* |    2150   | 3149  |
| *Pricey*  |    3150   |  '>'  |


**Conclusion** 

- Most part of Neighborhoods are Walker’s Paradise  
- Most part of Neighborhoods can offer maximum workplaces  (at least **50**)
- But Neighborhoods are expansive : major part of rents are more than **2950** until **4391**
- Finding a cheap rent is difficult and it costs beween **1625** until **2125**
- And finally Average mean is quite high and lower than the Median rent : **2798** v.s. **2950**

So it is why finding a job in the vicinity with a high walk score could allow to avoid spending money in cars.
This is the next step of analysis.

So the discrimination factor is clearly the price and the result of this study will be to provide the areas with the highest level of walkability, the highest density of venues and the lowest prices that could be resumed below :


  | Budget    | Very Walkable | Walker’s Paradise | Prices            |
  |-----------|:-------------:|:-----------------:|:-----------------:|
  |Cheap	  | 	  6 	  | 	  13	      | Y < 2150 	      |
  |Average	  | 	  5 	  |		  30 		  | 2149 < Y < 3150   |
  |___________|_______________|___________________|___________________|
  |Walk Score |   69 < X < 90 |		 X >= 90      |                   |



Now let's study the venues. 

###  e. Cartographic Localisation Exploration 

####  1. Target Neighborhoods in New York City 

##### Create a map of New York City with  neighborhoods and venues superimposed on top.

 Also, let's show borders of NYC boroughs on our map and a few circles indicating scores and prices with colors indicating the category of the place rated by the Walk Scores and Pricing !

In [ ]:
address = 'New York City, NY'
geo_agent = "ny_explorer"

geolocator = Nominatim(user_agent=geo_agent)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are : {}, {}.'.format(address, latitude, longitude))

In [ ]:
#Neighborhood 	1BR Budget 	Neighborhood Latitude 	Neighborhood Longitude 	Number Venues 	WS_mean WS_descr 	Index
# create map of New York using latitude and longitude values
#df_ws_viz
# add markers to map
chartreuse = 'chartreuse'         # 'Walker’s Paradise'/cheap
springgreen = 'springgreen'       # 'Very Walkable' /cheap
olive = 'olive'                   # 'Somewhat Walkable' 
darkgreen = 'darkgreen'           # 
red = 'red'                       #  pricey 
blue = 'blue'                     # 'Walker’s Paradise'/pricey
darkblue = 'darkblue'             # 'Very Walkable'/pricey
forestgreen = 'forestgreen'       # 'Very Walkable'/average
lightgreen = 'lightgreen'         # 'Walker’s Paradise'/ average
dimgrey = 'dimgrey'
gold ='gold'
palegoldenrod = 'palegoldenrod'
royalblue = 'royalblue'
navy = 'navy'


def get_nghb_color(ws, budget):
    color = 'blue'
    if (ws == 'Walker’s Paradise'):
        if (budget == 'Cheap'):
            color = chartreuse
        elif (budget == 'Average'):
            color = gold
        else:
            color = royalblue
    elif (ws == 'Very Walkable'):
        if (budget == 'Cheap'):
            color = olive
        elif (budget == 'Average'):
            color = palegoldenrod
        else:
            color = navy
    elif (ws == 'Somewhat Walkable'):
            color = dimgrey
    else: # car-dependant
        color = red
    return color

In [ ]:
map_newyork_v = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood,ws_descr, budget in zip(df_ws_viz['Neighborhood Latitude'], 
                                                   df_ws_viz['Neighborhood Longitude'],
                                                   df_ws_viz['Neighborhood'],
                                                   df_ws_viz['WS_descr'],
                                                   df_ws_viz['Budget']
                                                  ):
    label = '{}-{},{}'.format(neighborhood, ws_descr, budget)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=get_nghb_color(ws_descr, budget),
        fill=True,
        fill_color=get_nghb_color(ws_descr, budget), #'#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork_v)  
    
map_newyork_v

####  2. Target Places in New York City 

And now the same as above but visualizing venues locations with colors indicating the category of the place in function of the Walk Scores and Pricing !

In [ ]:
from matplotlib import colors as mcolors
#from folium.plugins import FloatImage

#from folium.plugins import FloatImage
#image_file = 'image.PNG'
#FloatImage(image_file, bottom=0, left=86).add_to(mymap)
map_newyork_v = folium.Map(location=[latitude, longitude], zoom_start=10)
#Neighborhood 	Neighborhood Latitude 	Neighborhood Longitude 	1BR 	Budget 	Venue 	Address 	Venue Latitude 	
# Venue Longitude 	Venue Category 	Walk_score 	Walk_descr            

for index, row in df_venues_sorted.iterrows(): # venues_score_df.iterrows():
#    if (index % 5 == 0):
        name_n = row['Neighborhood']
        name_v = row['Venue']
        lat_n = row['Neighborhood Latitude']
        lng_n = row['Neighborhood Longitude']
        lat_v = row['Venue Latitude']
        lng_v = row['Venue Longitude']
        adr_v = row['Address']
        ws    = row['Walk_descr']
        budget= row['Budget']
        br1   = row['1BR']
        
        color_v = get_nghb_color(ws, budget)
        label = name_v + '\n'+ adr_v + '\n' + '[' + ws + '/' + budget + ':' + str(int(br1)) + '$]'

        folium.CircleMarker([lat_v, lng_v], radius=3, color=color_v, fill=True,
                            popup=label, fill_color='blue', fill_opacity=0.6).add_to(map_newyork_v)

#map_newyork_v.get_root().html.add_child(folium.Element(legend_html)) 
map_newyork_v

In [ ]:
df_ws_viz_sorted =  df_ws_viz.copy().sort_values(['WS_mean', '1BR', 'Neighborhood Latitude', 'Neighborhood Longitude','Neighborhood'],
                                                ascending=[False,True,True,True,True]).reset_index(drop=True)
df_ws_viz_sorted.head()

####  3. Heatmap of Places in Walk Scored Areas in New York City 

##### Create a Choropleth heatmap of neighborhoods rated by Walk scores in New York City with  venues superimposed on top.

with colors indicating the category of the places in relative to the Walk Scores !

In [ ]:
ny_geo = r'new-york-city-boroughs.geojson' # geojson file
NYC_loc = [40.7308619,-73.9871558]

#df_ny = df_ws_viz.copy() #pd.read_csv('NYC_venues_walkability_0_3513.csv')
# create a plain NYC map

NYC_loc = [40.7308619,-73.9871558]
ny_map = folium.Map(location=[40.7308619,-73.9871558], zoom_start=10, tiles='Mapbox Bright')
ny_map.choropleth(
    geo_data=ny_geo,
    data=df_ws_viz_sorted,
    columns=['Neighborhood', 'WS_mean'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Walk scores neighborhood'
)
for index, row in df_venues_sorted.iterrows(): # venues_score_df.iterrows():
    #if (index % 5 == 0):
        name_n = row['Neighborhood']
        name_v = row['Venue']
        lat_n = row['Neighborhood Latitude']
        lng_n = row['Neighborhood Longitude']
        lat_v = row['Venue Latitude']
        lng_v = row['Venue Longitude']
        adr_v = row['Address']
        ws    = row['Walk_descr']
        budget= row['Budget']
        br1   = row['1BR']
        
        color_v = get_nghb_color(ws, budget)
        label = name_v + '\n'+ adr_v + '\n' + '[' + ws + '/' + budget + ':' + str(int(br1)) + '$]'

        folium.CircleMarker([lat_v, lng_v], radius=3, color=color_v, fill=True,
                            popup=label, fill_color='blue', fill_opacity=0.6).add_to(ny_map)
# display map
ny_map


In [ ]:

folium.__version__


####  4. Heatmap of Places in Rental Price-based Rated Areas  in New York City 

#### Create a heatmap of neighborhoods of New York City with  venues superimposed on top.

with colors indicating the category of the places relative to Pricing !

In [ ]:
# Try to load from local file system in case we did this before
filename = 'NYC_venues_selected.csv'

def load_selected_venues(df_venues):
    filename = 'NYC_venues_selected.csv'
    fromFile = False
    try:
        df_venues_sorted_limit = pd.read_csv(filename)
        df_venues_sorted_limit.drop(df.columns[[0]], axis=1, inplace=True)
        # Reset order of categorical columns 
        df_venues_sorted['Walk_descr']= df_venues_sorted['Walk_score'].apply(lambda x: getWalkLabel(x))
        df_venues_sorted.Walk_descr = set_col_cat(df_venues_sorted.Walk_descr, ws_cat)

        df_venues_sorted.Budget = df_venues_sorted['1BR'].apply(lambda x: rescaleBudgetLabel(x))
        df_venues_sorted.Budget = set_col_cat(df_venues_sorted.Budget)
        print('NYC Neighborhoods Venues Selected loaded from file :', 
              filename)
        fromFile = True
    except:
        pass

    # If not already loaded : use the Foursquare API to get the data
    if not fromFile:
        print (filename, ' : Data file for selected venues not existing ! Build it from df_venues')
        df_venues_sorted = df_venues.copy()
        df_venues_sorted['Walk_descr']= df_venues_sorted['Walk_score'].apply(lambda x: getWalkLabel(x))
        df_venues_sorted.Walk_descr = set_col_cat(df_venues_sorted.Walk_descr, ws_cat)

        df_venues_sorted.Budget = df_venues_sorted['1BR'].apply(lambda x: rescaleBudgetLabel(x))
        df_venues_sorted.Budget = set_col_cat(df_venues_sorted.Budget)

        #filter the wanted venues 
        df_venues_sorted_limit = df_venues_sorted[(df_venues_sorted['Walk_score'] >= 70)] # and (df_venues_sorted['1BR'] <= 3150)]
        df_venues_sorted_limit = df_venues_sorted_limit[df_venues_sorted_limit['1BR'] < 3150]

        print ('df_venues_sorted_limit shape ', df_venues_sorted_limit.shape)
        # Sort according our criteria Walk_decr and not Walk_score because 
        df_venues_sorted_limit = df_venues_sorted_limit.sort_values(['Walk_descr','1BR', 'Walk_score','Budget'],
                                                                      ascending=[False,True,False,True]).reset_index(drop=True)
        # Export the dataframe values to a local file  
        #print(df)
        df_venues_sorted_limit.to_csv(filename)
        print ('Selected Venues Data file created : ', filename, '-> shape = ', df_venues_sorted_limit.shape)
    return df_venues_sorted_limit

df_venues_sorted_limit = load_selected_venues(df_venues)
# New dataframe of selected venues in the correct order is available to be analysed and visualized 
# with normally in saved data file : 
# Selected Venues Data file created :  NYC_venues_selected.csv -> shape =  (1825, 12)
df_venues_sorted_limit.head(3)

In [ ]:
ny_geo = r'new-york-city-boroughs.geojson' # geojson file

#df_ny = df_ws_viz.copy() #pd.read_csv('NYC_venues_walkability_0_3513.csv')
# create a plain NYC map

NYC_loc = [40.7308619,-73.9871558]
ny_map_p = folium.Map(location=[40.7308619,-73.9871558], zoom_start=10, tiles='Mapbox Bright')
ny_map_p.choropleth(
    geo_data=ny_geo,
    data=df_ws_viz_sorted,
    columns=['Neighborhood', '1BR'], #['Neighborhood Latitude','Neighborhood Longitude','1BR','Walk_score'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Venues Prices Scored'
)
for index, row in df_venues_sorted.iterrows(): # venues_score_df.iterrows():
#    if (index % 5 == 0):
        name_n = row['Neighborhood']
        name_v = row['Venue']
        lat_n = row['Neighborhood Latitude']
        lng_n = row['Neighborhood Longitude']
        lat_v = row['Venue Latitude']
        lng_v = row['Venue Longitude']
        adr_v = row['Address']
        ws    = row['Walk_descr']
        budget= row['Budget']
        br1   = row['1BR']
        
        color_v = get_nghb_color(ws, budget)
        label = name_v + '\n'+ adr_v + '\n' + '[' + ws + '/' + budget + ':' + str(int(br1)) + '$]'

        folium.CircleMarker([lat_v, lng_v], radius=3, color=color_v, fill=True,
                            popup=label, fill_color='blue', fill_opacity=0.6).add_to(ny_map_p)
# display map
ny_map_p

Ok That looks good. Now we have about 2 thousands of venues (1825) localized in New York City and a good visualization of the places rated according to their Walkability or observe rental Pricing of 1 Bedroom apartment.

We can graphically say that several sections appear like (but not the only ones) :
- Good candidate places where rental price is 'cheap' and can live and work without owning a car 
    - Justice Ave (btwn Broadway & 52nd Ave), Elmhurst, NY 11373 (*Walker’s Paradise/Cheap 1900)
    - Woodhaven, NY 11421 (*Walker’s Paradise/Cheap:1625)
    - Ave (Jamaica Av), Richmond Hill, NY 11418 (*Walker’s Paradise/Cheap:1663)
    - Lefferts Blvd, South Ozone Park, NY 11420 (*Very Walkable/Cheap:1663)
    - etc...

but :
- Places to avoid if you have limited financial means even if walkable
    - Ave (at Metropolitan Ave), Brooklyn, NY 11211 (Walker’s Paradise/Pricey:3300)
    - Flatiron District New York, NY 10010 (Walker’s Paradise/Pricey:3552)
    -  Grand Ave, Maspeth, NY 11378 (Car-Dependant/Average:2331)
    - (Bld. 292), Brooklyn, NY 11205 (Car-Dependant/Pricey:3312)
    - etc...

And could continue along the list of adresses provided thanks to Foursquare for the venues, Walkscore for the pedestrian well-being and Renthop for the rental pricing.

Let us now cluster those locations to create a list of areas around venues gathering all our conditions i.e hihgly walkable with low rental prices and containing the maximum potential workplaces. The list issued from the processing ids the one that can be provided to stakeholders and usefull for people looking for places to live for cheap and with a good quality of life. 

Those zones, their centers and addresses will be the final result of our analysis. 


# Results

##### Let's perform now the last phase consisting in building the proposal to the stakeholders defining a set of neighborhoods and places in New York City combining the criteria we set up in the proble statement : place with no car-dependance, with high potential to find work places  and with the lowest rental prices.

This will lead us to define the clusters matching our criteria :
- First : at neighborhood level to have a high level map of areas of interest
- Second : at venues level to define areas adresses for a more accurate map

Let's retrieve and prepare the raw data needed for clustering matching the Walk Score, Number of venues and Rental Prices : 

- for neighborhoods 
- for venues 

####  1. Target Neighborhoods  Data Set

In [ ]:
dfd_ngh = pd.DataFrame([], index=budget_cat, columns=ws_cat)
dd = df_info_viz[['Budget', 'WS_descr', 'Number Venues']].copy()
dfd_ngh.apply(calc_total, args =('Count',dd), axis=1)

df_nac_summary =  dfd_ngh.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]
total_n = dfd_ngh.astype(float).sum().sum().astype(int)
print ('Total Neighborhoods = ', total_n)
total_ac_vp = df_nac_summary.astype(float).sum().astype(int)

print ('Total Neighborhoods = ', total_n, 'Neighborhoods fitting the criteria = ',total_ac_vp)

print('Total number of neighborhoods with Average or Cheap Rentals and more than Very Walkable = ', total_ac_vp.sum(),
      '\n -> Representing a rate of : ', str(round(total_ac_vp.sum()/total_n * 100).astype(int))+'%', )

df_nac_summary

Above are our neighborhoods set fitting the requirements distributed in 4 categories.

In [ ]:
# Build target neighborhoods raw data
#print(df_info_viz.dtypes, '\n', df_info_viz.shape)

df_neigh = df_info_viz.copy().reset_index(drop=True)
df_neigh = df_info_viz.copy()[df_info_viz['WS_descr'] >= 'Very Walkable']
df_neigh = df_neigh[df_neigh['Budget'] <= 'Average'] 

#print(df.dtypes, '\n', df.shape)
df_neigh = df_neigh.sort_values(['WS_mean','1BR', 'Number Venues'],ascending=[False,True,False]).reset_index(drop=True)
df_neigh

Well, we kept only our 45 over 81 of all neighborhoods prices are relatively affordable and with high Walkability.

Let's repeat the same calculus to explore now the number of venues for potential workplaces relative to Walkability scores.

####  2. Targeted Venues Data Set 

In [ ]:
dfd_nv = pd.DataFrame([], index=budget_cat, columns=ws_cat)

dfd_nv.apply(calc_total, args=('Number Venues', df_info_viz[['Budget', 'WS_descr', 'Number Venues']].copy()), axis=1)
df_ac =  dfd_nv.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]

total_v= dfd_nv.loc[['Cheap','Average','Pricey'],['Very Walkable','Walker’s Paradise']].astype(float).sum().sum().astype(int)
print ('Total Venues Very Walkable or Walker\'s Paradise = ', total_v)
total_ac_vp = df_ac.astype(float).sum().astype(int)
print('Total number of venues with lowest Rentals and more than Very Walkable = ', total_ac_vp.sum(),
      '\n -> Representing a rate of : ', str(round(total_ac_vp.sum()/total_v * 100).astype(int))+'%')

dfd_nv = dfd_nv.loc[['Average','Cheap'],['Very Walkable','Walker’s Paradise']]
dfd_nv

Now let's re-build the venues fitting the requirements

Above are our venues set fitting the requirements distributed also as the neighborhoods in 4 categories.

In [ ]:
# Try to load from local file system in case we did this before
filename = 'NYC_venues_selected.csv'

def load_selected_venues(df_venues):
    filename = 'NYC_venues_selected.csv'
    fromFile = False
    try:
        df_venues_sorted_limit = pd.read_csv(filename)
        df_venues_sorted_limit.drop(df.columns[[0]], axis=1, inplace=True)
        # Reset order of categorical columns 
        df_venues_sorted['Walk_descr']= df_venues_sorted['Walk_score'].apply(lambda x: getWalkLabel(x))
        df_venues_sorted.Walk_descr = set_col_cat(df_venues_sorted.Walk_descr, ws_cat)

        df_venues_sorted.Budget = df_venues_sorted['1BR'].apply(lambda x: rescaleBudgetLabel(x))
        df_venues_sorted.Budget = set_col_cat(df_venues_sorted.Budget)
        print('NYC Neighborhoods Venues Selected loaded from file :', 
              filename)
        fromFile = True
    except:
        pass

    # If not already loaded : use the Foursquare API to get the data
    if not fromFile:
        print (filename, ' : Data file for selected venues not existing ! Build it from df_venues')
        df_venues_sorted = df_venues.copy()
        df_venues_sorted['Walk_descr']= df_venues_sorted['Walk_score'].apply(lambda x: getWalkLabel(x))
        df_venues_sorted.Walk_descr = set_col_cat(df_venues_sorted.Walk_descr, ws_cat)

        df_venues_sorted.Budget = df_venues_sorted['1BR'].apply(lambda x: rescaleBudgetLabel(x))
        df_venues_sorted.Budget = set_col_cat(df_venues_sorted.Budget)

        #filter the wanted venues 
        df_venues_sorted_limit = df_venues_sorted[(df_venues_sorted['Walk_score'] >= 70)] # and (df_venues_sorted['1BR'] <= 3150)]
        df_venues_sorted_limit = df_venues_sorted_limit[df_venues_sorted_limit['1BR'] < 3150]

        print ('df_venues_sorted_limit shape ', df_venues_sorted_limit.shape)
        # Sort according our criteria Walk_decr and not Walk_score because 
        df_venues_sorted_limit = df_venues_sorted_limit.sort_values(['Walk_descr','1BR', 'Walk_score','Budget'],
                                                                      ascending=[False,True,False,True]).reset_index(drop=True)
        # Export the dataframe values to a local file  
        #print(df)
        df_venues_sorted_limit.to_csv(filename)
        print ('Selected Venues Data file created : ', filename, '-> shape = ', df_venues_sorted_limit.shape)
    return df_venues_sorted_limit

df_venues_sorted_limit = load_selected_venues(df_venues)
# New dataframe of selected venues in the correct order is available to be analysed and visualized 
# with normally in saved data file : 
# Selected Venues Data file created :  NYC_venues_selected.csv -> shape =  (1825, 12)
df_venues_sorted_limit.head(3)

In [ ]:
print ('df_venues_sorted_limit shape = ', df_venues_sorted_limit.shape)
df_vslc = df_venues_sorted_limit.groupby('Neighborhood',as_index=False).count().sort_values('Venue',ascending=True).reset_index(drop=True)
df_vslc['Number Venues'] = df_vslc['Venue']
print ('df_vslc grouped by venues number shape = ', df_vslc.shape)
df_vslc = df_vslc[['Neighborhood', 'Number Venues']]
df_vslc = df_vslc.sort_values('Number Venues',ascending=False).reset_index(drop=True)
df_vslc.head(3)

###  2. Clustering of Neighborhoods in New York City

####  a. Pre-processing

Before clustering we have to preprocess the data :
- Removing useless columns (categorical, string etc...)
- Normalizing data to be able to interpret features with different magnitudes and distributions equally... We will StandardScaler() to normalize our dataset.

In [ ]:
df = df_neigh.copy()

# Transform categorical in value number (rank)
#ws_cat = ['Car-Dependant', 'Somewhat Walkable', 'Very Walkable', 'Walker’s Paradise']
#budget_cat =['Cheap', 'Average', 'Pricey']
# Set 0 for lower values : 'Average' and 'Very Walkable', 1 otherwise
df['Budget_num'] = df['Budget'].apply(lambda x: convertCatToNum(x,'b')).astype(int)
df['WS_num'] = df['WS_descr'].apply(lambda x: convertCatToNum(x,'w')).astype(int)

# Now Removing useless columns : all non numerical
todrop = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Budget', 'WS_descr', 'Index']
for col in todrop:
    df.drop(col, axis=1, inplace=True)

print(df.dtypes,'   -> ', df.shape)
df.head(3)

##### Normalizing over the standard deviation

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.values[:,1:]
X = np.nan_to_num(X)
df_norm = StandardScaler().fit_transform(X)
print(X.shape, df_norm.shape)

##### Modeling Neighborhood Clusters with KMeans

In [ ]:
from sklearn.cluster import KMeans

# Should present 4 clusters
clusterNum = 4
k_means_ngh = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)
k_means_ngh.fit(X)
labels = k_means_ngh.labels_
print(len(labels), labels)

In [ ]:
df["Clus_lbl"] = labels
df

In [ ]:
df_neigh["Clus_lbl"] = labels
df_neigh.sort_values(['Clus_lbl','WS_mean', '1BR', 'Number Venues'],ascending=[True, False,True, False]).reset_index(drop=True)

Well, it seems that we dhave not exactly what we have found when manually sorting the dataset.
Except for Cluster 1 that seems rather close to previous results (same number as Average x Walker's Paradise). But we can see some Cheap 1BR Budget categorized inside this one.
That means that we need to tweak a little more the parameters.

I think the reason is the weight of each parameter : here all parameters have the same weight.
Likewise, we have also replaced the categorical attributes 'Budget' and 'WS_descr' by 0 or 1. 

Perhaps we should found most appropriate values.

##### Neighborhood Clusters Centroids

In [ ]:
df.groupby('Clus_lbl').mean()

In [ ]:
df.groupby('Clus_lbl').count()

##### Map of  Neighborhoods Localisation by Clusters

In [ ]:
map_newyork_cv = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood,ws_descr, budget, br1 in zip(df_neigh['Neighborhood Latitude'], 
                                                   df_neigh['Neighborhood Longitude'],
                                                   df_neigh['Neighborhood'],
                                                   df_neigh['WS_descr'],
                                                   df_neigh['Budget'],
                                                   df_neigh['1BR']
                                                  ):
    label = '{}-{},{}:${} '.format(neighborhood, ws_descr, budget, br1)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color=get_nghb_color(ws_descr, budget),
        fill=True,
        fill_color='#3186cc', #get_nghb_color(ws_descr, budget), #'#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork_cv)  
    
map_newyork_cv

###  3. Clustering of Target Places by Venues in New York City 

In this last step we will repeat the same processing as performed above but this time this will relate to the venues.

Like above, we repeat the same steps before clustering. We have to preprocess the data :
- Removing useless columns (categorical, string etc...)
- Normalizing data to be able to interpret features with different magnitudes and distributions equally... We will StandardScaler() to normalize our dataset.

In [ ]:
df_v = df_venues_sorted_limit.copy()
#print(df_v.shape, df_v.dtypes)
df_v.head(2)

In [ ]:
# Transform categorical in value number (rank)
df_venues = df_v.copy()

#ws_cat = ['Car-Dependant', 'Somewhat Walkable', 'Very Walkable', 'Walker’s Paradise']
#budget_cat =['Cheap', 'Average', 'Pricey']
# Set 0 for lower values : 'Average' and 'Very Walkable', 1 otherwise
df_v['Budget_num'] = df_v['Budget'].apply(lambda x: convertCatToNum(x,'b')).astype(int)
df_v['WS_num'] = df_v['Walk_descr'].apply(lambda x: convertCatToNum(x,'w')).astype(int)

# Now Removing useless columns : all non numerical
# Neighborhood, Neighborhood Latitude, Neighborhood Longitude,Budget,Venue,Address,
# Venue Latitude 	Venue Longitude, Venue Category, Walk_descr
todrop = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Budget',
          'Venue','Address','Venue Latitude','Venue Longitude','Venue Category',
          'Walk_descr']

for col in todrop:
    df_v.drop(col, axis=1, inplace=True)

print(df_v.dtypes,'   -> ', df_v.shape)
df_v.head(3)

##### Normalizing over the standard deviation

In [ ]:
from sklearn.preprocessing import StandardScaler

X_v = df_v.values[:,1:]
X_v = np.nan_to_num(X_v)
df_norm_v = StandardScaler().fit_transform(X_v)
print(X_v.shape, df_norm_v.shape)

##### Modeling Neighborhood Clusters with KMeans

In [ ]:
from sklearn.cluster import KMeans

# Should present 4 clusters
clusterNum = 4
k_means_v = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)
k_means_v.fit(X_v)
labels_v = k_means_v.labels_
print(len(labels_v), labels_v)

In [ ]:
df_venues['Clus_lbl'] = labels_v
df_venues.head(3)

##### Neighborhood Clusters Centroids

In [ ]:
df_venues.groupby('Clus_lbl').mean()

In [ ]:
df_venues.groupby('Clus_lbl').count()

Well, it seems that we have the same behaviour observed for the neighborhood. Well same causes same consequences !

The results are not exactly what we have found when manually sorting the dataset.

That means that we really need to tweak more the parameters.

Perhaps we have to found most appropriate values. This should be done in a future work of course !

#####  Heatmap of neighborhoods clusters rated by Walk scores in New York City  with  venues superimposed on top.

And colors indicating the category of the places in relative to the Walk Scores !

In [ ]:
ny_geo = r'new-york-city-boroughs.geojson' # geojson file
# create a plain NYC map

NYC_loc = [40.7308619,-73.9871558]
ny_map_v = folium.Map(location=[40.7308619,-73.9871558], zoom_start=10, tiles='Mapbox Bright')
ny_map_v.choropleth(
    geo_data=ny_geo,
    data=df_ws_viz_sorted,
    columns=['Neighborhood', 'WS_mean'], #['Neighborhood Latitude','Neighborhood Longitude','1BR','Walk_score'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Walk Score Neighborhood Scale'
)
for index, row in df_venues.iterrows(): # venues_score_df.iterrows():
        name_n = row['Neighborhood']
        name_v = row['Venue']
        lat_n = row['Neighborhood Latitude']
        lng_n = row['Neighborhood Longitude']
        lat_v = row['Venue Latitude']
        lng_v = row['Venue Longitude']
        adr_v = row['Address']
        ws    = row['Walk_descr']
        budget= row['Budget']
        br1   = row['1BR']
        
        color_v = get_nghb_color(ws, budget)
        label = name_v + '\n'+ adr_v + '\n' + '[' + ws + '/' + budget + ':' + str(int(br1)) + '$]'

        folium.CircleMarker([lat_v, lng_v], radius=3, color=color_v, fill=True,
                            popup=label, fill_color='blue', fill_opacity=0.6).add_to(ny_map_v)
# display map
ny_map_v


#### Conclusion

We have defined a visual tool to allow people to easily search for an apartment to rent according to the criteria they want, based on :
 - pedestrian possibilities or walkability of a place within a radius of 500 m from the center of visualized circle
 - the opportunity degree to find a job and 
 - the amount of rent they want to dedicate. 
 
These maps of neighborhoods and places can ease the life of the stakeholders and answer to the question we defined at the problenm statement.

Of course this is a first draft that needs improvement to be more accurate.

# Results

Using the data available for a non professionnal project, so limited in number and quality, our analysis shows that we were able to retrieve several thousands of basic information on places (a number of about 3500 venues in New York) represented about only 80 neighborhoods over the 306 existing in New York City.

So, unfortunately these data does not cover the totality of New City.

The most part of the work performed here was to collect and process the data so that it could be used for analysis.

Based on this data set, we can extract the points of interest gathering all the criteria defined in the requirements exposed in the problem statement.

We filtered out the elements that constitute a good starting point to consider the areas fitting the following criteria :

- pedestrian possibilities or walkability of a place within a radius of 500 m from the center of visualized circle
- opportunity degree to find a job and
- needed amount of money to rent a 1 bedroom apartment to locate they want to dedicate.

# Conclusion

We finally succeded in partitioning manually the candidate places in 4 categories.

We tried to reproduce this categorisation using clustering methods. The obtained cluters do not fit exactly those obtained manually.

The clustering parameters should be tweaked more carefully. In particular, it should be possible to try and adjust to reflect the importance given to each criterion which can change the nature and composition of the clusters.

But it is a promising track. 

We could already propose a list of adresses of places, summarized on a interactive maps of New York City that would meet the needs of customers.

This work could interest directly categories of professionnals like online rental agencies or recommendation tools.

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).